# RAPTOR

---

<br>

Run command:

```python pyraptor/gtfs/timetable.py -i "data/input/NL-gtfs" -o "data/output" -d "20220517" -a NS --icd```

## Setup

In [1]:
input_folder: str = "data/input/NL-gtfs"
output_folder: str = "data/output"
departure_date: str = "20220517"
agencies: list[str] = ['NS']
icd_fix: bool = False

## Imports

In [2]:
from collections import defaultdict
import pandas as pd
import os

In [3]:
from pyraptor.dao import write_timetable
from pyraptor.util import mkdir_if_not_exists, str2sec, TRANSFER_COST
from pyraptor.model.structures import (
    Timetable,
    Stop,
    Stops,
    Trip,
    Trips,
    TripStopTime,
    TripStopTimes,
    Station,
    Stations,
    Routes,
    Transfer,
    Transfers,
)

# Timetable

- Task: Duplicate the TimeTable (GTFS and RAPTOR) produced in [pyraptor/gtfs/timetable.py](pyraptor/gtfs/timetable.py)
- Purpose: Reference

## GtfsTimetable

In [4]:
# @dataclass
class GtfsTimetable:
    """Gtfs Timetable data"""

    trips = None
    calendar = None
    stop_times = None
    stops = None


### calculate_icd_fare

In [5]:
def calculate_icd_fare(trip: Trip, stop: Stop, stations: Stations) -> int:
    """Get supplemental fare for ICD"""
    fare = 0
    if 900 <= trip.hint <= 1099:
        if (
            trip.hint % 2 == 0 and stop.station == stations.get("Schiphol Airport")
        ) or (
            trip.hint % 2 == 1 and stop.station == stations.get("Rotterdam Centraal")
        ):
            fare = 1.67
        else:
            fare = 0
    return fare

## read_gtfs_timetable

The following is the implementation of the `read_gtfs_timetable` function (detailed in the dropdown below)

<details>
<summary><h4 style="color: green;">read_gtfs_timetable:<h4></summary>
<body> </body>
<p></p>

```python
def read_gtfs_timetable(
    input_folder: str, departure_date: str, agencies: list[str]
) -> GtfsTimetable:
    """Extract operators from GTFS data"""

    # logger.info("Read GTFS data")


    # AGENCY.TXT
    # logger.debug("Read Agencies")

    # Load dataframe and slice to specified agencies
    agencies_df = pd.read_csv(os.path.join(input_folder, "agency.txt"))
    agencies_df = agencies_df.loc[agencies_df["agency_name"].isin(agencies)][
        ["agency_id", "agency_name"]
    ]
    agency_ids = agencies_df.agency_id.values
    assert len(agency_ids) > 0
    # logger.debug(f"Agencies found: {agency_ids}")


    # ROUTES.TXT
    # logger.debug("Read Routes")

    # Get routes by agency
    routes = pd.read_csv(os.path.join(input_folder, "routes.txt"))
    routes = routes[routes.agency_id.isin(agency_ids)]
    routes = routes[
        ["route_id", "agency_id", "route_short_name", "route_long_name", "route_type"]
    ]
    assert len(routes) > 0
    # logger.debug(f"Routes found: {len(routes)} routes")


    # TRIPS.TXT
    # logger.debug("Read Trips")

    # Get trips in routes
    trips = pd.read_csv(os.path.join(input_folder, "trips.txt"))
    trips = trips[trips.route_id.isin(routes.route_id.values)]
    trips = trips[
        [
            "route_id",
            "service_id",
            "trip_id",
            "trip_short_name",
            "trip_long_name",
        ]
    ]
    trips["trip_short_name"] = trips["trip_short_name"].astype(int)
    assert len(trips) > 0
    # logger.debug(f"Trips found: {len(trips)} trips")


    # CALENDAR.TXT
    # logger.debug("Read Calendar")

    # Get calendar from service_id of trips (correct dates)
    calendar = pd.read_csv(
        os.path.join(input_folder, "calendar_dates.txt"), dtype={"date": str}
    )
    assert departure_date in calendar.date.values
    # logger.debug(f"Departure date ({departure_date}) in GTFS ({input_folder + 'calendar_dates.txt'})")
    calendar = calendar[calendar.service_id.isin(trips.service_id.values)]

    # Add date to trips and filter on departure date
    trips = trips.merge(calendar[["service_id", "date"]], on="service_id")
    trips = trips[trips.date == departure_date]
    assert len(trips) > 0
    # logger.debug(f"Trips on departure date found: {len(trips)} trips")


    # STOP_TIMES.TXT
    # logger.debug("Read Stop Times")

    # Get stop times of trips
    stop_times = pd.read_csv(
        os.path.join(input_folder, "stop_times.txt"), dtype={"stop_id": str}
    )
    stop_times = stop_times[stop_times.trip_id.isin(trips.trip_id.values)]
    stop_times = stop_times[
        [
            "trip_id",
            "stop_sequence",
            "stop_id",
            "arrival_time",
            "departure_time",
        ]
    ]
    # Convert times to seconds past midnight
    stop_times["arrival_time"] = stop_times["arrival_time"].apply(str2sec)
    stop_times["departure_time"] = stop_times["departure_time"].apply(str2sec)
    assert len(stop_times) > 0
    # logger.debug(f"Stop times found: {len(stop_times)} stop times")


    # PLATFORMS (STOPS.TXT)
    # NOTE: Platforms are found from unique `stop_id`s
    # logger.debug("Read Stops")

    stops_full = pd.read_csv(
        os.path.join(input_folder, "stops.txt"), dtype={"stop_id": str}
    )
    stops = stops_full.loc[
        stops_full["stop_id"].isin(stop_times.stop_id.unique())
    ].copy()

    # Read stopareas, i.e. stations
    stopareas = stops["parent_station"].unique()
    # stops = stops.append(.copy())
    stops = pd.concat([stops, stops_full.loc[stops_full["stop_id"].isin(stopareas)]])

    # stops["zone_id"] = stops["zone_id"].str.replace("IFF:", "").str.upper()
    stops["stop_code"] = stops.stop_code.str.upper()
    stops = stops[
        [
            "stop_id",
            "stop_name",
            "parent_station",
            "platform_code",
        ]
    ]

    # Filter out the general station codes
    stops = stops.loc[~stops.parent_station.isna()]
    assert len(stops) > 0
    stops = stops.loc[~stops.parent_station.isna()]

    gtfs_timetable = GtfsTimetable()
    gtfs_timetable.trips = trips
    gtfs_timetable.stop_times = stop_times
    gtfs_timetable.stops = stops

    return gtfs_timetable
```
    
</details>

### agency.txt

In [6]:
# AGENCY.TXT
# logger.debug("Read Agencies")

# Load dataframe and slice to specified agencies
agencies_df = pd.read_csv(os.path.join(input_folder, "agency.txt"))
agencies_df = agencies_df.loc[agencies_df["agency_name"].isin(agencies)][
    ["agency_id", "agency_name"]
]
agency_ids = agencies_df.agency_id.values
assert len(agency_ids) > 0
# logger.debug(f"Agencies found: {agency_ids}")

agency_ids

array(['IFF:NS'], dtype=object)

### routes.txt

In [7]:
# ROUTES.TXT
# logger.debug("Read Routes")

# Get routes by agency
routes = pd.read_csv(os.path.join(input_folder, "routes.txt"))
routes = routes[routes.agency_id.isin(agency_ids)]
routes = routes[
    ["route_id", "agency_id", "route_short_name", "route_long_name", "route_type"]
]
assert len(routes) > 0
# logger.debug(f"Routes found: {len(routes)} routes")

routes

,route_id,agency_id,route_short_name,route_long_name,route_type
4,67394,IFF:NS,Intercity,Den Haag Centraal <-> Eindhoven Centraal IC1100,2
5,67395,IFF:NS,Intercity,Nachtnettrein Rotterdam Centraal <-> Eindhoven...,2
7,67399,IFF:NS,Intercity,Den Haag Centraal <-> Amersfoort Centraal IC2000,2
9,67400,IFF:NS,Intercity,Eindhoven Centraal <-> Venlo IC13500,2
10,67402,IFF:NS,Intercity,Nachtnettrein Utrecht Centraal <-> Rotterdam C...,2
...,...,...,...,...,...
2555,81768,IFF:NS,Metro i.p.v. trein,Metro i.p.v. trein Amsterdam Sloterdijk <-> Am...,1
2556,81769,IFF:NS,Metro i.p.v. trein,Metro i.p.v. trein Amsterdam Sloterdijk <-> Am...,1
2557,81770,IFF:NS,Metro i.p.v. trein,Metro i.p.v. trein Amsterdam Sloterdijk <-> Am...,1
2558,81771,IFF:NS,Snelbus i.p.v. trein,Snelbus i.p.v. trein Amsterdam Sloterdijk <-> ...,3


### trips.txt

In [8]:
# TRIPS.TXT
# logger.debug("Read Trips")

# Get trips in routes
trips = pd.read_csv(os.path.join(input_folder, "trips.txt"))
trips = trips[trips.route_id.isin(routes.route_id.values)]
trips = trips[
    [
        "route_id",
        "service_id",
        "trip_id",
        "trip_short_name",
        "trip_long_name",
    ]
]
trips["trip_short_name"] = trips["trip_short_name"].astype(int)
assert len(trips) > 0
# logger.debug(f"Trips found: {len(trips)} trips")


# CALENDAR.TXT
# logger.debug("Read Calendar")

# Get calendar from service_id of trips (correct dates)
calendar = pd.read_csv(
    os.path.join(input_folder, "calendar_dates.txt"), dtype={"date": str}
)
assert departure_date in calendar.date.values
# logger.debug(f"Departure date ({departure_date}) in GTFS ({input_folder + 'calendar_dates.txt'})")
calendar = calendar[calendar.service_id.isin(trips.service_id.values)]

# Add date to trips and filter on departure date
trips = trips.merge(calendar[["service_id", "date"]], on="service_id")
trips = trips[trips.date == departure_date]
assert len(trips) > 0
# logger.debug(f"Trips on departure date found: {len(trips)} trips")

trips

,route_id,service_id,trip_id,trip_short_name,trip_long_name,date
0,52984,1553,140721829,3984,Intercity,20220517
208,52984,1553,140721828,3983,Intercity,20220517
416,52984,1553,140721830,3988,Intercity,20220517
624,41216,1553,140726469,7054,Sprinter,20220517
832,52984,1553,140721827,3980,Intercity,20220517
...,...,...,...,...,...,...
813243,17601,461,140728963,11728,Intercity,20220517
813437,17601,282,140728985,11741,Intercity,20220517
813472,17601,490,140729063,11788,Intercity,20220517
813586,46092,699,140729127,14610,Sprinter,20220517


### calendar.txt

In [9]:
calendar

,service_id,date,exception_type
0,1,20220517,1
1,2,20220517,1
2,2,20220617,1
3,2,20220624,1
4,2,20220701,1
...,...,...,...
316950,5034,20221106,1
316951,5034,20221113,1
316952,5034,20221120,1
316953,5034,20221127,1


### stop_times.txt

In [10]:
# STOP_TIMES.TXT
# logger.debug("Read Stop Times")

# Get stop times of trips
stop_times = pd.read_csv(
    os.path.join(input_folder, "stop_times.txt"), dtype={"stop_id": str}
)
stop_times = stop_times[stop_times.trip_id.isin(trips.trip_id.values)]
stop_times = stop_times[
    [
        "trip_id",
        "stop_sequence",
        "stop_id",
        "arrival_time",
        "departure_time",
    ]
]
# Convert times to seconds past midnight
stop_times["arrival_time"] = stop_times["arrival_time"].apply(str2sec)
stop_times["departure_time"] = stop_times["departure_time"].apply(str2sec)
assert len(stop_times) > 0
# logger.debug(f"Stop times found: {len(stop_times)} stop times")

stop_times

,trip_id,stop_sequence,stop_id,arrival_time,departure_time
102569,140721829,7,2324446,79440,79440
104228,144756632,1,2422049,87840,87840
104229,144756632,3,2324441,88260,88320
104284,147754397,1,2422147,42480,42480
104285,147754397,26,2323857,48480,48480
...,...,...,...,...,...
14378909,151771259,2,2422877,30960,30960
14378910,151771261,2,2422877,34560,34560
14378911,151771264,2,2422877,39960,39960
14378912,151771270,2,2422877,50760,50760


### stops.txt
(Platforms)

In [11]:
# PLATFORMS (STOPS.TXT)
# NOTE: Platforms are found from unique `stop_id`s
# logger.debug("Read Stops")

stops_full = pd.read_csv(
    os.path.join(input_folder, "stops.txt"), dtype={"stop_id": str}
)
stops = stops_full.loc[
    stops_full["stop_id"].isin(stop_times.stop_id.unique())
].copy()

# Read stopareas, i.e. stations
stopareas = stops["parent_station"].unique()
# stops = stops.append(.copy())
stops = pd.concat([stops, stops_full.loc[stops_full["stop_id"].isin(stopareas)]])

# stops["zone_id"] = stops["zone_id"].str.replace("IFF:", "").str.upper()
stops["stop_code"] = stops.stop_code.str.upper()
stops = stops[
    [
        "stop_id",
        "stop_name",
        "parent_station",
        "platform_code",
    ]
]

# Filter out the general station codes
stops = stops.loc[~stops.parent_station.isna()]
assert len(stops) > 0
stops = stops.loc[~stops.parent_station.isna()]

stops

,stop_id,stop_name,parent_station,platform_code
19217,2324635,Vlissingen,stoparea:18286,3
19753,2422053,Dordrecht,stoparea:18308,2
19756,2324687,Wijhe,stoparea:18087,2
19757,2422302,Brummen,stoparea:18048,2
19763,2423677,Goes,stoparea:17800,2
...,...,...,...,...
31373,2324347,Rotterdam Centraal,stoparea:17894,4
31423,2324681,Wolfheze,stoparea:18082,3
32192,2324311,Roosendaal,stoparea:17963,3b
32226,2323098,Arnhem Velperpoort,stoparea:17881,2


### GTFSTimetable

In [12]:
gtfs_timetable = GtfsTimetable()
gtfs_timetable.trips = trips
gtfs_timetable.stop_times = stop_times
gtfs_timetable.stops = stops

In [13]:
### gtfs_to_pyraptor_timetable()

def gtfs_to_pyraptor_timetable(
    gtfs_timetable: GtfsTimetable, icd_fix: bool = False
) -> Timetable:
    """
    Convert timetable for usage in Raptor algorithm.
    """
    # logger.info("Convert GTFS timetable to timetable for PyRaptor algorithm")

    # Stations and stops, i.e. platforms
    # logger.debug("Add stations and stops")

    stations = Stations()
    stops = Stops()

    gtfs_timetable.stops.platform_code = gtfs_timetable.stops.platform_code.fillna("?")

    for s in gtfs_timetable.stops.itertuples():
        station = Station(s.stop_name, s.stop_name)
        station = stations.add(station)

        stop_id = f"{s.stop_name}-{s.platform_code}"
        stop = Stop(s.stop_id, stop_id, station, s.platform_code)

        station.add_stop(stop)
        stops.add(stop)

    # Stop Times
    stop_times = defaultdict(list)
    for stop_time in gtfs_timetable.stop_times.itertuples():
        stop_times[stop_time.trip_id].append(stop_time)

    # Trips and Trip Stop Times
    # logger.debug("Add trips and trip stop times")

    trips = Trips()
    trip_stop_times = TripStopTimes()

    for trip_row in gtfs_timetable.trips.itertuples():
        trip = Trip()
        trip.hint = trip_row.trip_short_name  # i.e. treinnummer
        trip.long_name = trip_row.trip_long_name  # e.g., Sprinter

        # Iterate over stops
        sort_stop_times = sorted(
            stop_times[trip_row.trip_id], key=lambda s: int(s.stop_sequence)
        )
        for stopidx, stop_time in enumerate(sort_stop_times):
            # Timestamps
            dts_arr = stop_time.arrival_time
            dts_dep = stop_time.departure_time

            # Trip Stop Times
            stop = stops.get(stop_time.stop_id)

            # GTFS files do not contain ICD supplement fare, so hard-coded here
            fare = calculate_icd_fare(trip, stop, stations) if icd_fix is True else 0
            trip_stop_time = TripStopTime(trip, stopidx, stop, dts_arr, dts_dep, fare)

            trip_stop_times.add(trip_stop_time)
            trip.add_stop_time(trip_stop_time)

        # Add trip
        if trip:
            trips.add(trip)

    # Routes
    # logger.debug("Add routes")

    routes = Routes()
    for trip in trips:
        routes.add(trip)

    # Transfers
    # logger.debug("Add transfers")

    transfers = Transfers()
    for station in stations:
        station_stops = station.stops
        station_transfers = [
            Transfer(from_stop=stop_i, to_stop=stop_j, layovertime=TRANSFER_COST)
            for stop_i in station_stops
            for stop_j in station_stops
            if stop_i != stop_j
        ]
        for st in station_transfers:
            transfers.add(st)

    # Timetable
    timetable = Timetable(
        stations=stations,
        stops=stops,
        trips=trips,
        trip_stop_times=trip_stop_times,
        routes=routes,
        transfers=transfers,
    )
    timetable.counts()

    return timetable

In [14]:
# Not callable
# gtfs_timetable = read_gtfs_timetable(input_folder, departure_date, agencies)

timetable = gtfs_to_pyraptor_timetable(gtfs_timetable, icd_fix)

2022-06-10 16:35:52.593 | DEBUG    | pyraptor.model.structures:counts:41 - Counts:
2022-06-10 16:35:52.594 | DEBUG    | pyraptor.model.structures:counts:42 - Stations   : 248
2022-06-10 16:35:52.594 | DEBUG    | pyraptor.model.structures:counts:43 - Routes     : 899
2022-06-10 16:35:52.595 | DEBUG    | pyraptor.model.structures:counts:44 - Trips      : 4305
2022-06-10 16:35:52.595 | DEBUG    | pyraptor.model.structures:counts:45 - Stops      : 736
2022-06-10 16:35:52.595 | DEBUG    | pyraptor.model.structures:counts:46 - Stop Times : 42508
2022-06-10 16:35:52.596 | DEBUG    | pyraptor.model.structures:counts:47 - Transfers  : 3036


In [15]:
timetable

Timetable(stations=<Stations(n_stations=248)>, stops=Stops(n_stops=736), trips=Trips(n_trips=4305), trip_stop_times=TripStoptimes(n_tripstoptimes=42508), routes=Routes(n_routes=899), transfers=Transfers(n_transfers=3036))

# Structures

1. Timetable
1. Stop
1. Stops
1. Station
1. Stations
1. TripStopTime
1. TripStopTimes
1. Trip
1. Trips
1. Route
1. Routes
1. Transfer
1. Transfers
1. Leg
1. Label
1. Bag
1. Journey
1. pareto_set

### 1 - Timetable

Attributes:
1. stations: Stations
1. stops: Stops
1. trips: Trips
1. trip_stop_times: TripStopTimes
1. routes: Routes
1. transfers: Transfers

Methods:
1. counts

In [16]:
timetable.stations.get_stops('Rotterdam Centraal')

[Stop(Rotterdam Centraal-9 [2427778]),
 Stop(Rotterdam Centraal-14 [2427770]),
 Stop(Rotterdam Centraal-16 [2422141]),
 Stop(Rotterdam Centraal-2 [2427775]),
 Stop(Rotterdam Centraal-15 [2422140]),
 Stop(Rotterdam Centraal-13 [2427769]),
 Stop(Rotterdam Centraal-12 [2427768]),
 Stop(Rotterdam Centraal-6 [2324348]),
 Stop(Rotterdam Centraal-11 [2427767]),
 Stop(Rotterdam Centraal-7 [2427776]),
 Stop(Rotterdam Centraal-8 [2427777]),
 Stop(Rotterdam Centraal-3 [2324346]),
 Stop(Rotterdam Centraal-4 [2324347])]

### 2 - Stop

Attributes:
1. id = attr.ib(default=None)
1. name = attr.ib(default=None)
1. station: Station = attr.ib(default=None)
1. platform_code = attr.ib(default=None) # I want to see how this is handled
1. index = attr.ib(default=None)

Methods:
1. \_\_hash\_\_
1. \_\_eq\_\_
1. \_\_repr\_\_

In [17]:
Stop_eg = timetable.stations.get_stops('Rotterdam Centraal')[0]
Stop_eg1 = timetable.stations.get_stops('Rotterdam Centraal')[1]
print(f"Stop_eg: {Stop_eg}")
print()
print(f"Attributes:")
print(f"\t id: {Stop_eg.id}")
print(f"\t name: {Stop_eg.name}")
print(f"\t station: {Stop_eg.station}")
print(f"\t platform_code: {Stop_eg.platform_code}")
print(f"\t index: {Stop_eg.index}")
print()
print(f"Methods:")
print(f"\t __hash__: {Stop_eg.__hash__()}")
print(f"\t __eq__: {Stop_eg.__eq__(Stop_eg1)}")
print(f"\t __repr__: {Stop_eg.__repr__()}")

Stop_eg: Stop(Rotterdam Centraal-9 [2427778])

Attributes:
	 id: 2427778
	 name: Rotterdam Centraal-9
	 station: Station(Rotterdam Centraal)
	 platform_code: 9
	 index: 41

Methods:
	 __hash__: 3946690538992712051
	 __eq__: False
	 __repr__: Stop(Rotterdam Centraal-9 [2427778])


#### NOTE:
- `Stop.name`s are `Station.name`s with a suffix of `-{Stop.platform_code}`

### 3 - Stops

Attributes:
1. set_idx
1. set_index
1. last_index

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter_\_\_
1. get
1. get_by_index
1. add

In [18]:
Stops_eg = timetable.stops
print(f"Stops_eg: {Stops_eg}")
print()
print("Attributes:")
print(f"\t set_idx['2324635']: {Stops_eg.set_idx['2324635']}")
print(f"\t set_index[1]: {Stops_eg.set_index[1]}")
print(f"\t last_index: {Stops_eg.last_index}")
print()
print("Methods:")
print(f"\t __repr__: {Stops_eg.__repr__()}")
print(f"\t __getitem__: {Stops_eg.__getitem__(stop_id='2427778')}")
print(f"\t __len__: {Stops_eg.__len__()}")
print(f"\t __iter__: {Stops_eg.__iter__()}")
print(f"\t get: {Stops_eg.get(stop_id='2427778')}")
print(f"\t get_by_index: {Stops_eg.get_by_index(stop_index=700)}")

Stops_eg: Stops(n_stops=736)

Attributes:
	 set_idx['2324635']: Stop(Vlissingen-3 [2324635])
	 set_index[1]: Stop(Vlissingen-3 [2324635])
	 last_index: 737

Methods:
	 __repr__: Stops(n_stops=736)
	 __getitem__: Stop(Rotterdam Centraal-9 [2427778])
	 __len__: 736
	 __iter__: <dict_valueiterator object at 0x127e7a930>
	 get: Stop(Rotterdam Centraal-9 [2427778])
	 get_by_index: Stop(Lelystad Centrum-1 [2422110])


### 4 - Station

Attributes:
1. id
1. name
1. stops

Methods:
1. \_\_hash\_\_
1. \_\_eq\_\_
1. \_\_repr\_\_
1. add_stop

In [19]:
Station_eg = timetable.stations.__getitem__(station_id='Rotterdam Centraal')
Station_eg1 = timetable.stations.__getitem__(station_id='Lelystad Centrum')
print(f"Station_eg: {Station_eg}")
print()
print("Attributes:")
print(f"\t id: {Station_eg.id}")
print(f"\t name: {Station_eg.name}")
print(f"\t stops: {Station_eg.stops}")
print()
print(f"Methods:")
print(f"\t __hash__: {Station_eg.__hash__()}")
print(f"\t __eq__: {Station_eg.__eq__(Station_eg1)}")
print(f"\t __repr__: {Station_eg.__repr__()}")

Station_eg: Station(Rotterdam Centraal)

Attributes:
	 id: Rotterdam Centraal
	 name: Rotterdam Centraal
	 stops: [Stop(Rotterdam Centraal-9 [2427778]), Stop(Rotterdam Centraal-14 [2427770]), Stop(Rotterdam Centraal-16 [2422141]), Stop(Rotterdam Centraal-2 [2427775]), Stop(Rotterdam Centraal-15 [2422140]), Stop(Rotterdam Centraal-13 [2427769]), Stop(Rotterdam Centraal-12 [2427768]), Stop(Rotterdam Centraal-6 [2324348]), Stop(Rotterdam Centraal-11 [2427767]), Stop(Rotterdam Centraal-7 [2427776]), Stop(Rotterdam Centraal-8 [2427777]), Stop(Rotterdam Centraal-3 [2324346]), Stop(Rotterdam Centraal-4 [2324347])]

Methods:
	 __hash__: 5951843383199407511
	 __eq__: False
	 __repr__: Station(Rotterdam Centraal)


### 5 - Stations

Attributes:
1. set_idx

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add
1. get
1. get_stops

In [20]:
Stations_eg = timetable.stations
Stations_eg1 = timetable.stations
print(f"Stations_eg: {Stations_eg}")
print()
print("Attributes:")
print(f"\t set_idx['Vlissingen']: {Stations_eg.set_idx['Vlissingen']}")
print()
print("Methods:")
print(f"\t __repr__: {Stations_eg.__repr__()}")
print(f"\t __getitem__: {Stations_eg.__getitem__(station_id='Rotterdam Centraal')}")
print(f"\t __len__: {Stations_eg.__len__()}")
print(f"\t __iter__: {Stations_eg.__iter__()}")
print(f"\t get: {Stations_eg.get(station='Rotterdam Centraal')}")
print(f"\t get_stops: {Stations_eg.get_stops(station_name='Rotterdam Centraal')}")

Stations_eg: <Stations(n_stations=248)>

Attributes:
	 set_idx['Vlissingen']: Station(Vlissingen)

Methods:
	 __repr__: <Stations(n_stations=248)>
	 __getitem__: Station(Rotterdam Centraal)
	 __len__: 248
	 __iter__: <dict_valueiterator object at 0x127e7acf0>
	 get: Station(Rotterdam Centraal)
	 get_stops: [Stop(Rotterdam Centraal-9 [2427778]), Stop(Rotterdam Centraal-14 [2427770]), Stop(Rotterdam Centraal-16 [2422141]), Stop(Rotterdam Centraal-2 [2427775]), Stop(Rotterdam Centraal-15 [2422140]), Stop(Rotterdam Centraal-13 [2427769]), Stop(Rotterdam Centraal-12 [2427768]), Stop(Rotterdam Centraal-6 [2324348]), Stop(Rotterdam Centraal-11 [2427767]), Stop(Rotterdam Centraal-7 [2427776]), Stop(Rotterdam Centraal-8 [2427777]), Stop(Rotterdam Centraal-3 [2324346]), Stop(Rotterdam Centraal-4 [2324347])]


### 6 - TripStopTime

Attributes:
1. trip: Trip
1. stopidx
1. stop
1. dts_arr
1. dts_dep
1. fare
1. **occupancy**

Methods:
1. \_\_hash\_\_
1. \_\_repr\_\_

In [21]:
TripStopTime_eg = timetable.trip_stop_times.stop_trip_idx[Stop_eg][0]
TripStopTime_eg1 = timetable.trip_stop_times.stop_trip_idx[Stop_eg1][0]
print(f"TripStopTime_eg: {TripStopTime_eg}")
print()
print("Attributes:")
print(f"\t trip: {TripStopTime_eg.trip}")
print(f"\t stopidx: {TripStopTime_eg.stopidx}")
print(f"\t stop: {TripStopTime_eg.stop}")
print(f"\t dts_arr: {TripStopTime_eg.dts_arr}")
print(f"\t dts_dep: {TripStopTime_eg.dts_dep}")
print(f"\t fare: {TripStopTime_eg.fare}")
print(f"\t occupancy: {TripStopTime_eg.occupancy}")
print()
print("Methods:")
print(f"\t __hash__: {TripStopTime_eg.__hash__} ")
print(f"\t hash: {TripStopTime_eg.hash()}")
print(f"\t __repr__: {TripStopTime_eg.__repr__()}")

TripStopTime_eg: TripStopTime(trip_id=('1178:',)(50,), stopidx=3, stop_id=2427778, dts_arr=81960, dts_dep=82080, fare=0, occupancy=0.0)

Attributes:
	 trip: Trip(hint=1178, stop_times=7)
	 stopidx: 3
	 stop: Stop(Rotterdam Centraal-9 [2427778])
	 dts_arr: 81960
	 dts_dep: 82080
	 fare: 0
	 occupancy: 0.0

Methods:
	 __hash__: None 
	 hash: 6093587361206976270
	 __repr__: TripStopTime(trip_id=('1178:',)(50,), stopidx=3, stop_id=2427778, dts_arr=81960, dts_dep=82080, fare=0, occupancy=0.0)


In [22]:
# from pyraptor.model.structures import TripStopTime

print(f"\t occupancy: {TripStopTime_eg.occupancy}")
TripStopTime_eg.update_occupancy()
print(f"\t occupancy: {TripStopTime_eg.occupancy}")


	 occupancy: 0.0
	 occupancy: 1.0


### 7 - TripStopTimes

Attributes:
1. set_idx: Dict[Tuple[Trip, int], TripStopTime]
1. stop_trip_idx: Dict[Stop, List[TripStopTime]]

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add
1. get_trip_stop_times_in_range
1. get_earliest_trip
1. get_earliest_trip_stop_time

In [23]:
TripStopTimes_eg = timetable.trip_stop_times
print(f"TripStopTimes_eg: {TripStopTimes_eg}")
print()
print("Attributes:")
print(f"\t len(set_idx): {len(TripStopTimes_eg.set_idx)}")
print(f"\t len(stop_trip_idx): {len(TripStopTimes_eg.stop_trip_idx)}")
print()
# print(f"Methods: \n __getitem__: {TripStopTimes_eg.__getitem__()}")
print(f"Methods:")
print(f" \t __repr__: {TripStopTimes_eg.__repr__()}")
print(f" \t __len__: {TripStopTimes_eg.__len__()}")
print(f" \t __iter__: {TripStopTimes_eg.__iter__()}")
print(f" \t get_trip_stop_times_in_range: {TripStopTimes_eg.get_trip_stop_times_in_range(stops=timetable.stations.get_stops('Rotterdam Centraal'), dep_secs_min=str2sec('10:00:00'), dep_secs_max=str2sec('10:04:00'))}")
print(f" \t get_earliest_trip: {TripStopTimes_eg.get_earliest_trip(stop=Stop_eg, dep_secs=str2sec('10:00'))}")
print(f" \t get_earliest_trip_stop_time: {TripStopTimes_eg.get_earliest_trip_stop_time(stop=timetable.stations.get_stops('Rotterdam Centraal')[0], dep_secs=str2sec('10:00:00'))}")

TripStopTimes_eg: TripStoptimes(n_tripstoptimes=42508)

Attributes:
	 len(set_idx): 42508
	 len(stop_trip_idx): 736

Methods:
 	 __repr__: TripStoptimes(n_tripstoptimes=42508)
 	 __len__: 42508
 	 __iter__: <dict_valueiterator object at 0x1359c1350>
 	 get_trip_stop_times_in_range: [TripStopTime(trip_id=('4031:',)(545,), stopidx=22, stop_id=2422141, dts_arr=36240, dts_dep=36240, fare=0, occupancy=0.0), TripStopTime(trip_id=('3237:',)(2088,), stopidx=0, stop_id=2427778, dts_arr=36060, dts_dep=36060, fare=0, occupancy=0.0)]
 	 get_earliest_trip: Trip(hint=1178, stop_times=7)
 	 get_earliest_trip_stop_time: TripStopTime(trip_id=('1178:',)(50,), stopidx=3, stop_id=2427778, dts_arr=81960, dts_dep=82080, fare=0, occupancy=1.0)


### 8 - Trip

Attributes:
1. id
1. stop_times: list
1. stop_times_index: dict
1. hint: str
1. long_name: str

Methods:
1. \_\_hash\_\_
1. \_\_eq_\_
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. trip_stop_ids
1. add_stop_time
1. get_stop
1. get_fare

In [24]:
print(f"Stop_eg: {Stop_eg}")
Trip_eg = TripStopTimes_eg.get_earliest_trip(stop=Stop_eg, dep_secs=str2sec('10:00'))
Trip_eg1 = TripStopTimes_eg.get_earliest_trip(stop=Stop_eg, dep_secs=str2sec('11:00'))
print(f"Trip_eg: {Trip_eg}")
print()
print("Attributes:")
print(f"\t id: {Trip_eg.id}")
print(f"\t stop_times: {Trip_eg.stop_times}")
print(f"\t stop_times_index: {Trip_eg.stop_times_index}")
print(f"\t hint: {Trip_eg.hint}")
print(f"\t long_name: {Trip_eg.long_name}")
print()
print("Methods:")
print(f"\t __hash__: {Trip_eg.__hash__()}")
print(f"\t __eq__: {Trip_eg.__eq__(trip=Trip_eg1)}")
print(f"\t __repr__: {Trip_eg.__repr__()}")
print(f"\t __getitem__: {Trip_eg.__getitem__(n=1)}")
print(f"\t __len__: {Trip_eg.__len__()}")
print(f"\t __iter__: {Trip_eg.__iter__()}")
print(f"\t trip_stop_ids: {Trip_eg.trip_stop_ids()}")
print(f"\t get_stop: {Trip_eg.get_stop(Stop_eg)}")
print(f"\t get_fare: {Trip_eg.get_fare(Stop_eg)}")
print(f"\t get_stop_occupancy: {Trip_eg.get_stop_occupancy(Stop_eg)}")


Stop_eg: Stop(Rotterdam Centraal-9 [2427778])
Trip_eg: Trip(hint=1178, stop_times=7)

Attributes:
	 id: 50
	 stop_times: [TripStopTime(trip_id=('1178:',)(50,), stopidx=0, stop_id=2323599, dts_arr=78240, dts_dep=78240, fare=0, occupancy=0.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=1, stop_id=2324474, dts_arr=79500, dts_dep=79740, fare=0, occupancy=0.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=2, stop_id=2422045, dts_arr=80460, dts_dep=80580, fare=0, occupancy=0.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=3, stop_id=2427778, dts_arr=81960, dts_dep=82080, fare=0, occupancy=1.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=4, stop_id=2422319, dts_arr=82740, dts_dep=82740, fare=0, occupancy=0.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=5, stop_id=2422082, dts_arr=83160, dts_dep=83280, fare=0, occupancy=0.0), TripStopTime(trip_id=('1178:',)(50,), stopidx=6, stop_id=2423310, dts_arr=83520, dts_dep=83520, fare=0, occupancy=0.0)]
	 stop_times_index: {Stop(Eindhoven Ce

In [25]:
print(f"type(Trip_eg): {type(Trip_eg)}")
print(f"Trip_eg.get_stop_occupancy(Stop_eg): {Trip_eg.get_stop_occupancy(Stop_eg)}")
print(f"Trip_eg.update_occupancy(Stop_eg): {Trip_eg.update_occupancy(Stop_eg)}")
print(f"Trip_eg.get_stop_occupancy(Stop_eg): {Trip_eg.get_stop_occupancy(Stop_eg)}")

type(Trip_eg): <class 'pyraptor.model.structures.Trip'>
Trip_eg.get_stop_occupancy(Stop_eg): 1.0
Trip_eg.update_occupancy(Stop_eg): None
Trip_eg.get_stop_occupancy(Stop_eg): 2.0


### 9 - Trips

Attributes:
1. set_idx
1. last_id

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add

In [26]:
Trips_eg = timetable.trips
print(f"Trips_eg: {Trips_eg}")
print()
print("Attributes:")
print(f"\t set_idx[1]: {Trips_eg.set_idx[1]}")
print(f"\t last_id: {Trips_eg.last_id}")
print()
print("Methods:")
print(f"\t __repr__: {Trips_eg.__repr__()}")
print(f"\t __getitem__: {Trips_eg.__getitem__(trip_id=1)}")
print(f"\t __len__: {Trips_eg.__len__()}")
print(f"\t __iter__: {Trips_eg.__iter__()}")

Trips_eg: Trips(n_trips=4305)

Attributes:
	 set_idx[1]: Trip(hint=3984, stop_times=2)
	 last_id: 4306

Methods:
	 __repr__: Trips(n_trips=4305)
	 __getitem__: Trip(hint=3984, stop_times=2)
	 __len__: 4305
	 __iter__: <dict_valueiterator object at 0x2a7605df0>


### 10 - Route

Attributes:
1. set_idx: dict
1. set_stops_idx: dict
1. stop_to_routes: dict(list)
1. stop_order: int

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add_trip
1. add_stop
1. stop_index
1. earliest_trip
1. earliest_trip_stop_time

In [27]:
Route_eg = timetable.routes.__getitem__(route_id=1)
Route_eg1 = timetable.routes.__getitem__(route_id=2)
print(f"Route_eg: {Route_eg}")
print()
print("Attributes:")
print(f"\t trips: {Route_eg.trips}")
print(f"\t stops: {Route_eg.stops}")
print(f"\t stop_order: {Route_eg.stop_order}")
print()
print("Methods:")
print(f"\t __hash__: {Route_eg.__hash__()}")
print(f"\t __eq__: {Route_eg.__eq__(trip=Route_eg1)}")
print(f"\t __repr__: {Route_eg.__repr__()}")
print(f"\t __getitem__: {Route_eg.__getitem__(n=1)}")
print(f"\t __len__: {Route_eg.__len__()}")
print(f"\t __iter__: {Route_eg.__iter__()}")
print(f"\t stop_index: {Route_eg.stop_index(stop=timetable.stations.get_stops('Heerlen')[0])}")
print(f"\t earliest_trip: {Route_eg.earliest_trip(dts_arr=str2sec('10:00:00'), stop=timetable.stations.get_stops('Heerlen')[0])}")
print(f"\t earliest_trip_stop_time: {Route_eg.earliest_trip_stop_time(dts_arr=str2sec('10:00:00'), stop=timetable.stations.get_stops('Heerlen')[0])}")

Route_eg: Route(id=1, trips=4)

Attributes:
	 trips: [Trip(hint=3984, stop_times=2), Trip(hint=3988, stop_times=2), Trip(hint=3980, stop_times=2), Trip(hint=3992, stop_times=2)]
	 stops: [Stop(Heerlen-5 [2323896]), Stop(Sittard-20 [2324446])]
	 stop_order: {Stop(Heerlen-5 [2323896]): 0, Stop(Sittard-20 [2324446]): 1}

Methods:
	 __hash__: 1
	 __eq__: False
	 __repr__: Route(id=1, trips=4)
	 __getitem__: Trip(hint=3988, stop_times=2)
	 __len__: 4
	 __iter__: <list_iterator object at 0x2bb321090>
	 stop_index: 0
	 earliest_trip: Trip(hint=3980, stop_times=2)
	 earliest_trip_stop_time: TripStopTime(trip_id=('3980:',)(5,), stopidx=0, stop_id=2323896, dts_arr=74940, dts_dep=74940, fare=0, occupancy=0.0)


### 11 - Routes

Attributes:
1. id
1. trips: list
1. stops: list
1. stop_order: dict

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add
1. get_routes_of_stop

In [28]:
Routes_eg = timetable.routes
print(f"Routes_eg: {Routes_eg}")
print()
print("Attributes:")
print(f"\t set_idx[1]: {Routes_eg.set_idx[1]}")
print(f"\t set_stops_idx[('2323896', '2324446')]: {Routes_eg.set_stops_idx[('2323896', '2324446')]}")
print(f"\t stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]: {Routes_eg.stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]}")
print(f"\t last_id: {Routes_eg.last_id}")
print()
print("Methods:")
print(f"\t __repr__: {Routes_eg.__repr__()}")
print(f"\t __getitem__: {Routes_eg.__getitem__(route_id=1)}")
print(f"\t __len__: {Routes_eg.__len__()}")
print(f"\t __iter__: {Routes_eg.__iter__()}")
print(f"\t get_routes_of_stop: {Routes_eg.get_routes_of_stop(stop=timetable.stations.get_stops('Heerlen')[0])}")

Routes_eg: Routes(n_routes=899)

Attributes:
	 set_idx[1]: Route(id=1, trips=4)
	 set_stops_idx[('2323896', '2324446')]: Route(id=1, trips=4)
	 stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]: [Route(id=1, trips=4), Route(id=2, trips=5), Route(id=26, trips=1), Route(id=198, trips=1), Route(id=211, trips=14), Route(id=237, trips=2), Route(id=296, trips=17), Route(id=371, trips=3), Route(id=428, trips=1), Route(id=466, trips=1), Route(id=528, trips=1), Route(id=534, trips=2), Route(id=558, trips=1), Route(id=564, trips=2), Route(id=571, trips=3), Route(id=701, trips=5), Route(id=723, trips=1), Route(id=853, trips=1)]
	 last_id: 900

Methods:
	 __repr__: Routes(n_routes=899)
	 __getitem__: Route(id=1, trips=4)
	 __len__: 899
	 __iter__: <dict_valueiterator object at 0x2a7604400>
	 get_routes_of_stop: [Route(id=1, trips=4), Route(id=2, trips=5), Route(id=26, trips=1), Route(id=198, trips=1), Route(id=211, trips=14), Route(id=237, trips=2), Route(id=296, trips=17), Route(id=371, 

### 12 - Transfer

Attributes:
1. id
1. from_stop: Stop
1. to_stop: Stop
1. layovertime: int

Methods:
1. \_\_hash_\_
1. \_\_eq\_\_
1. \_\_repr\_\_

In [29]:
Transfer_eg = timetable.transfers.__getitem__(transfer_id=1)
Transfer_eg1 = timetable.transfers.__getitem__(transfer_id=2)
print(f"Transfer_eg: {Transfer_eg}")
print()
print("Attributes:")
print(f"\t id: {Transfer_eg.id}")
print(f"\t from_stop: {Transfer_eg.from_stop}")
print(f"\t to_stop: {Transfer_eg.to_stop}")
print(f"\t layovertime: {Transfer_eg.layovertime}")
print()
print("Methods:")
print(f"\t __hash__: {Transfer_eg.__hash__()}")
print(f"\t __eq__: {Transfer_eg.__eq__(trip=Transfer_eg1)}")
print(f"\t __repr__: {Transfer_eg.__repr__()}")

Transfer_eg: Transfer(from_stop=Stop(Vlissingen-3 [2324635]), to_stop=Stop(Vlissingen-1 [2324633]), layovertime=120)

Attributes:
	 id: 1
	 from_stop: Stop(Vlissingen-3 [2324635])
	 to_stop: Stop(Vlissingen-1 [2324633])
	 layovertime: 120

Methods:
	 __hash__: 1
	 __eq__: False
	 __repr__: Transfer(from_stop=Stop(Vlissingen-3 [2324635]), to_stop=Stop(Vlissingen-1 [2324633]), layovertime=120)


In [30]:
Routes_eg = timetable.routes
print(f"Routes_eg: {Routes_eg}")
print()
print("Attributes:")
print(f"\t set_idx[1]: {Routes_eg.set_idx[1]}")
print(f"\t set_stops_idx[('2323896', '2324446')]: {Routes_eg.set_stops_idx[('2323896', '2324446')]}")
print(f"\t stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]: {Routes_eg.stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]}")
print(f"\t last_id: {Routes_eg.last_id}")
print()
print("Methods:")
print(f"\t __repr__: {Routes_eg.__repr__()}")
print(f"\t __getitem__: {Routes_eg.__getitem__(route_id=1)}")
print(f"\t __len__: {Routes_eg.__len__()}")
print(f"\t __iter__: {Routes_eg.__iter__()}")
print(f"\t get_routes_of_stop: {Routes_eg.get_routes_of_stop(stop=timetable.stations.get_stops('Heerlen')[0])}")

Routes_eg: Routes(n_routes=899)

Attributes:
	 set_idx[1]: Route(id=1, trips=4)
	 set_stops_idx[('2323896', '2324446')]: Route(id=1, trips=4)
	 stop_to_routes[timetable.stations.get_stops('Heerlen')[0]]: [Route(id=1, trips=4), Route(id=2, trips=5), Route(id=26, trips=1), Route(id=198, trips=1), Route(id=211, trips=14), Route(id=237, trips=2), Route(id=296, trips=17), Route(id=371, trips=3), Route(id=428, trips=1), Route(id=466, trips=1), Route(id=528, trips=1), Route(id=534, trips=2), Route(id=558, trips=1), Route(id=564, trips=2), Route(id=571, trips=3), Route(id=701, trips=5), Route(id=723, trips=1), Route(id=853, trips=1)]
	 last_id: 900

Methods:
	 __repr__: Routes(n_routes=899)
	 __getitem__: Route(id=1, trips=4)
	 __len__: 899
	 __iter__: <dict_valueiterator object at 0x2a7606070>
	 get_routes_of_stop: [Route(id=1, trips=4), Route(id=2, trips=5), Route(id=26, trips=1), Route(id=198, trips=1), Route(id=211, trips=14), Route(id=237, trips=2), Route(id=296, trips=17), Route(id=371, 

### 13 - Transfers

Attributes:
1. set_idx: dict
1. stop_to_stop_idx = dict
1. last_id: int

Methods:
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_len\_\_
1. \_\_iter\_\_
1. add

In [31]:
# timetable.stations.get_stops('Vlissingen')[0], timetable.stations.get_stops('Vlissingen')[1]

In [32]:
Transfers_eg = timetable.transfers
print(f"Transfers_eg: {Transfers_eg}")
print()
print("Attributes:")
print(f"\t set_idx[1]: {Transfers_eg.set_idx[1]}")
print(f"\t stop_to_stop_idx[(timetable.stations.get_stops('Vlissingen')[0], timetable.stations.get_stops('Vlissingen')[1])]: {Transfers_eg.stop_to_stop_idx[(timetable.stations.get_stops('Vlissingen')[0], timetable.stations.get_stops('Vlissingen')[1])]}")
print(f"\t last_id: {Transfers_eg.last_id}")
print()
print("Methods:")
print(f"\t __repr__: {Transfers_eg.__repr__()}")
print(f"\t __getitem__: {Transfers_eg.__getitem__(transfer_id=1)}")
print(f"\t __len__: {Transfers_eg.__len__()}")
print(f"\t __iter__: {Transfers_eg.__iter__()}")

Transfers_eg: Transfers(n_transfers=3036)

Attributes:
	 set_idx[1]: Transfer(from_stop=Stop(Vlissingen-3 [2324635]), to_stop=Stop(Vlissingen-1 [2324633]), layovertime=120)
	 stop_to_stop_idx[(timetable.stations.get_stops('Vlissingen')[0], timetable.stations.get_stops('Vlissingen')[1])]: Transfer(from_stop=Stop(Vlissingen-3 [2324635]), to_stop=Stop(Vlissingen-1 [2324633]), layovertime=120)
	 last_id: 3037

Methods:
	 __repr__: Transfers(n_transfers=3036)
	 __getitem__: Transfer(from_stop=Stop(Vlissingen-3 [2324635]), to_stop=Stop(Vlissingen-1 [2324633]), layovertime=120)
	 __len__: 3036
	 __iter__: <dict_valueiterator object at 0x13592e890>


### 14 - Leg

Attributes:
1. from_stop: Stop
1. to_stop: Stop
1. trip: Trip
1. earliest_arrival_time: int
1. fare: int
1. n_trips: int

Methods:
1. criteria
1. dep
1. arr
1. is_transfer
1. is_compatible_before
1. to_dict

### 15 - Label

Attributes:
1. earliest_arrival_time: int
1. fare: int  # total fare
1. trip: Trip  # trip to take to obtain travel_time and fare
1. from_stop: Stop  # stop to hop-on the trip
1. n_trips: int
1. infinite: bool

Methods:
1. criteria
1. update
1. update_trip

### 16 - Bag

Attributes:
1. N/A

Methods:
1. \_\_len\_\_
1. \_\_repr\_\_
1. add
1. merge
1. labels_with_trips
1. earliest_arrival

### 17 - Journey

Attributes:
1. N/A

Methods:
1. \_\_len\_\_
1. \_\_repr\_\_
1. \_\_getitem\_\_
1. \_\_iter\_\_
1. \_\_lt\_\_
1. number_of_trips
1. prepend_leg
1. remove_transfer_leg
1. is_valid
1. from_stop
1. to_stop
1. fare
1. dep
1. arr
1. travel_time
1. dominates
1. print
1. to_list

# Create timetable

In [33]:
# !python pyraptor/gtfs/timetable.py -i "data/input/NL-gtfs" -o "data/output" -d "20220517" -a NS --icd

# Raptor

- Task: Duplicate the RaptorAlgorithm found in [pyraptor/model/raptor.py](pyraptor/model/raptor.py)
- Purpose: Reference, implementation of occupancy

## Imports

In [34]:
from __future__ import annotations
from typing import List, Tuple, Dict
from dataclasses import dataclass
from copy import deepcopy

from loguru import logger

from pyraptor.dao.timetable import Timetable
from pyraptor.model.structures import Stop, Trip, Route, Leg, Journey
from pyraptor.util import LARGE_NUMBER, TRANSFER_TRIP

In [35]:
# !pip install git+https://github.com/yunusskeete/pyraptor.git --quiet

## Label

In [36]:
@dataclass
class Label:
    """Label"""

    earliest_arrival_time: int = LARGE_NUMBER
    trip: Trip = None  # trip to take to obtain earliest_arrival_time
    from_stop: Stop = None  # stop at which we hop-on trip with trip

    def update(self, earliest_arrival_time=None, trip=None, from_stop=None):
        """Update"""
        if earliest_arrival_time is not None:
            self.earliest_arrival_time = earliest_arrival_time
        if trip is not None:
            self.trip = trip
        if from_stop is not None:
            self.from_stop = from_stop

    def is_dominating(self, other: Label):
        """Dominates other label"""
        return self.earliest_arrival_time <= other.earliest_arrival_time

    def __repr__(self) -> str:
        return f"Label(earliest_arrival_time={self.earliest_arrival_time}, trip={self.trip}, from_stop={self.from_stop})"


## RaptorAlgorithm

In [37]:
class RaptorAlgorithm:
    """RAPTOR Algorithm"""

    def __init__(self, timetable: Timetable):
        self.timetable = timetable
        self.bag_star = None

    def run(self, from_stops, dep_secs, rounds) -> Dict[int, Dict[Stop, Label]]:
        """Run Round-Based Algorithm"""

        # Initialize empty bag of labels, i.e. B_k(p) = Label() for every k and p
        bag_round_stop: Dict[int, Dict[Stop, Label]] = {}
        for k in range(0, rounds + 1):
            bag_round_stop[k] = {}
            for p in self.timetable.stops:
                bag_round_stop[k][p] = Label()

        # Initialize bag with earliest arrival tiems
        self.bag_star = {}
        for p in self.timetable.stops:
            self.bag_star[p] = Label()

        # Initialize bag with start node taking DEP_SECS seconds to reach
        logger.debug(f"Starting from Stop IDs: {str(from_stops)}")
        marked_stops = []
        for from_stop in from_stops:
            # Update Label with departure time
            # bag_round_stop[0][from_stop].update(earliest_arrival_time=dep_secs, fare_addition=None, from_stop=None)
            bag_round_stop[0][from_stop].update(dep_secs, None, None)
            # self.bag_star[from_stop].update(earliest_arrival_time=dep_secs, fare_addition=None, from_stop=None)
            self.bag_star[from_stop].update(dep_secs, None, None)
            marked_stops.append(from_stop)

        # Run rounds
        for k in range(1, rounds + 1):
            logger.info(f"Analyzing possibilities round {k}")
            bag_round_stop[k] = deepcopy(bag_round_stop[k - 1])

            # Get list of stops to evaluate in the process
            logger.debug(f"Stops to evaluate count: {len(marked_stops)}")

            # Get marked route stops - A list of routes serving marked stops from previous round, i.e. Q
            route_marked_stops = self.accumulate_routes(marked_stops)



            ### HERE ###########################################################



            # Update time to stops calculated based on stops reachable
            bag_round_stop, marked_trip_stops = self.traverse_routes(
                bag_round_stop, k, route_marked_stops
            )
            logger.debug(f"{len(marked_trip_stops)} reachable stops added")

            # Add footpath transfers and update
            bag_round_stop, marked_transfer_stops = self.add_transfer_time(
                bag_round_stop, k, marked_trip_stops
            )
            logger.debug(f"{len(marked_transfer_stops)} transferable stops added")

            marked_stops = set(marked_trip_stops).union(marked_transfer_stops)
            logger.debug(f"{len(marked_stops)} stops to evaluate in next round")

        return bag_round_stop

    def accumulate_routes(self, marked_stops: List[Stop]) -> List[Tuple[Route, Stop]]:
        """Accumulate routes serving marked stops from previous round, i.e. Q"""
        route_marked_stops = {}  # i.e. Q
        for marked_stop in marked_stops:
            routes_serving_stop = self.timetable.routes.get_routes_of_stop(marked_stop)
            for route in routes_serving_stop:
                # Check if new_stop is before existing stop in Q

                # Get value for key (route), return None if key not in dictionary
                current_stop_for_route = route_marked_stops.get(route, None)  # p'
                # If we haven't marked this route stop
                # or
                # if current_stop_for_route comes after the marked stop???
                if (current_stop_for_route is None) or (
                    route.stop_index(current_stop_for_route)
                    > route.stop_index(marked_stop)
                ):
                    route_marked_stops[route] = marked_stop
        route_marked_stops = [(r, p) for r, p in route_marked_stops.items()]

        return route_marked_stops

    def traverse_routes(
        self,
        bag_round_stop: Dict[int, Dict[Stop, Label]],
        k: int,
        route_marked_stops: List[Tuple[Route, Stop]],
    ) -> Tuple:
        """
        Iterator through the stops reachable and add all new reachable stops
        by following all trips from the reached stations. Trips are only followed
        in the direction of travel and beyond already added points.

        :param bag_round_stop: Bag per round per stop
        :param k: current round
        :param route_marked_stops: list of marked (route, stop) for evaluation
        """
        logger.debug(f"Traverse routes for round {k}")

        bag_round_stop = deepcopy(bag_round_stop)
        new_stops = []
        n_evaluations = 0
        n_improvements = 0

        # For each route
        for (marked_route, marked_stop) in route_marked_stops:

            # Current trip for this marked stop
            current_trip = None

            # Iterate over all stops after current stop within the current route
            current_stop_index = marked_route.stop_index(marked_stop)
            remaining_stops_in_route = marked_route.stops[current_stop_index:]
            boarding_stop = None

            for current_stop_index, current_stop in enumerate(remaining_stops_in_route):
                # Can the label be improved in this round?
                n_evaluations += 1

                # t != _|_
                if current_trip is not None:
                    # Arrival time at stop, i.e. arr(current_trip, next_stop)
                    new_arrival_time = current_trip.get_stop(current_stop).dts_arr
                    best_arrival_time = self.bag_star[
                        current_stop
                    ].earliest_arrival_time

                    if new_arrival_time < best_arrival_time:
                        # Update arrival by trip, i.e.
                        #   t_k(next_stop) = t_arr(t, pi)
                        #   t_star(p_i) = t_arr(t, pi)

                        # Update Label
                        bag_round_stop[k][current_stop].update(
                            new_arrival_time, current_trip, boarding_stop
                        )
                        self.bag_star[current_stop].update(
                            new_arrival_time, current_trip, boarding_stop
                        )

                        # Logging
                        n_improvements += 1
                        new_stops.append(current_stop)

                # Can we catch an earlier trip at p_i
                # if tau_{k-1}(next_stop) <= tau_dep(t, next_stop)
                previous_earliest_arrival_time = bag_round_stop[k][
                    current_stop
                ].earliest_arrival_time
                earliest_trip_stop_time = marked_route.earliest_trip_stop_time(
                    previous_earliest_arrival_time, current_stop
                )
                if (
                    earliest_trip_stop_time is not None
                    and previous_earliest_arrival_time
                    <= earliest_trip_stop_time.dts_dep
                ):
                    current_trip = earliest_trip_stop_time.trip
                    boarding_stop = current_stop

        logger.debug(f"- Evaluations    : {n_evaluations}")
        logger.debug(f"- Improvements   : {n_improvements}")

        return bag_round_stop, new_stops

    def add_transfer_time(
        self,
        bag_round_stop: Dict[int, Dict[Stop, Label]],
        k: int,
        marked_stops: List[Stop],
    ) -> Tuple:
        """
        Add transfers between platforms.

        :param bag_round_stop: Label per round per stop
        :param k: current round
        :param marked_stops: list of marked stops for evaluation
        """

        new_stops = []

        # Add in transfers to other platforms
        for current_stop in marked_stops:
            other_station_stops = [
                st for st in current_stop.station.stops if st != current_stop
            ]

            time_sofar = bag_round_stop[k][current_stop].earliest_arrival_time
            for arrive_stop in other_station_stops:
                new_earliest_arrival = time_sofar + self.get_transfer_time(
                    current_stop, arrive_stop
                )
                previous_earliest_arrival = self.bag_star[
                    arrive_stop
                ].earliest_arrival_time

                # Domination criteria
                if new_earliest_arrival < previous_earliest_arrival:
                    bag_round_stop[k][arrive_stop].update(
                        new_earliest_arrival,
                        TRANSFER_TRIP,
                        current_stop,
                    )
                    self.bag_star[arrive_stop].update(
                        new_earliest_arrival, TRANSFER_TRIP, current_stop
                    )
                    new_stops.append(arrive_stop)

        return bag_round_stop, new_stops

    def get_transfer_time(self, stop_from: Stop, stop_to: Stop) -> int:
        """
        Calculate the transfer time from a stop to another stop (usually at one station)
        """
        transfers = self.timetable.transfers
        return transfers.stop_to_stop_idx[(stop_from, stop_to)].layovertime



## best_stop_at_target_station

In [38]:
def best_stop_at_target_station(to_stops: List[Stop], bag: Dict[Stop, Label]) -> Stop:
    """
    Find the destination Stop with the shortest distance.
    Required in order to prevent adding travel time to the arrival time.
    """
    final_stop = 0
    distance = LARGE_NUMBER
    for stop in to_stops:
        if bag[stop].earliest_arrival_time < distance:
            distance = bag[stop].earliest_arrival_time
            final_stop = stop
    return final_stop

## reconstruct_journey

In [39]:
def reconstruct_journey(destination: Stop, bag: Dict[Stop, Label]) -> Journey:
    """Construct journey for destination from values in bag.
    
    MY THINKING:
    -   Bags detail earliest_arrival_time, fare, trip, from_stop
    -   When querying raptor, we reconstruct the journey from the destination stop
        with a bag of best_labels (the earliest arrival at destination stop)
    -   From our destination, we iterate backwards.
        We accumulate the best trips till we get to our origin
        (bag details best ways of getting to stops from origin)
    
    """

    # Create journey with list of legs
    jrny = Journey()
    to_stop = destination
    while to_stop is not None:
        # print(f"to_stop: {to_stop}")
        # Get the trip start stop
        from_stop = bag[to_stop].from_stop
        # Get the trip
        bag_to_stop = bag[to_stop]
        leg = Leg(
            from_stop, to_stop, bag_to_stop.trip, bag_to_stop.earliest_arrival_time
        )
        # print(f"bag_to_stop: {bag_to_stop}")
        jrny = jrny.prepend_leg(leg)
        to_stop = from_stop

    jrny = jrny.remove_transfer_legs()

    return jrny

## is_dominated

In [40]:
def is_dominated(original_journey: List[Leg], new_journey: List[Leg]) -> bool:
    """Check if new journey is dominated by another journey"""
    # None if first journey
    if not original_journey:
        return False

    # No improvement
    if original_journey == new_journey:
        return True

    def depart(jrny: List[Leg]) -> int:
        depart_leg = jrny[0] if jrny[0].trip is not None else jrny[1]
        return depart_leg.dep

    def arrival(jrny: List[Leg]) -> int:
        return jrny[-1].arr

    original_depart = depart(original_journey)
    new_depart = depart(new_journey)

    original_arrival = arrival(original_journey)
    new_arrival = arrival(new_journey)

    # Is dominated, strictly better in one criteria and not worse in other
    return (
        True
        if (original_depart >= new_depart and original_arrival < new_arrival)
        or (original_depart > new_depart and original_arrival <= new_arrival)
        else False
    )

## query_raptor

### Imports

In [41]:
import argparse
from typing import Dict

from loguru import logger

from pyraptor.dao.timetable import read_timetable
from pyraptor.model.structures import Journey, Station, Timetable
# from pyraptor.model.raptor import (
#     RaptorAlgorithm,
#     reconstruct_journey,
#     best_stop_at_target_station,
# )
from pyraptor.util import str2sec

### parse_arguments

<details>
    <summary><h4 style="color: green;">parse_arguments():<h4></summary>
<body> </body>
<p></p>

```python
def parse_arguments():
    """Parse arguments"""
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "-i",
        "--input",
        type=str,
        default="data/output",
        help="Input directory",
    )
    parser.add_argument(
        "-or",
        "--origin",
        type=str,
        default="Hertogenbosch ('s)",
        help="Origin station of the journey",
    )
    parser.add_argument(
        "-d",
        "--destination",
        type=str,
        default="Rotterdam Centraal",
        help="Destination station of the journey",
    )
    parser.add_argument(
        "-t", "--time", type=str, default="08:35:00", help="Departure time (hh:mm:ss)"
    )
    parser.add_argument(
        "-r",
        "--rounds",
        type=int,
        default=5,
        help="Number of rounds to execute the RAPTOR algorithm",
    )
    arguments = parser.parse_args()
    return arguments
```

</details>

### main

In [42]:
def main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
):
    """Run RAPTOR algorithm"""

    logger.debug("Input directory     : {}", input_folder)
    logger.debug("Origin station      : {}", origin_station)
    logger.debug("Destination station : {}", destination_station)
    logger.debug("Departure time      : {}", departure_time)
    logger.debug("Rounds              : {}", str(rounds))

    # timetable = read_timetable(input_folder)

    logger.info(f"Calculating network from: {origin_station}")

    # Departure time seconds
    dep_secs = str2sec(departure_time)
    logger.debug("Departure time (s.)  : " + str(dep_secs))

    # Find route between two stations
    journey_to_destinations = run_raptor(
        timetable,
        origin_station,
        dep_secs,
        rounds,
    )

    # Print journey to destination
    journey_to_destinations[destination_station].print(dep_secs=dep_secs)

### run_raptor

In [43]:
def run_raptor(
    timetable: Timetable,
    origin_station: str,
    dep_secs: int,
    rounds: int,
) -> Dict[Station, Journey]:
    """
    Run the Raptor algorithm.

    :param timetable: timetable
    :param origin_station: Name of origin station
    :param dep_secs: Time of departure in seconds
    :param rounds: Number of iterations to perform
    """

    # Get stops for origin and all destinations
    from_stops = timetable.stations.get(origin_station).stops
    destination_stops = {
        st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
    }
    destination_stops.pop(origin_station, None)

    # Run Round-Based Algorithm
    raptor = RaptorAlgorithm(timetable)
    bag_round_stop = raptor.run(from_stops, dep_secs, rounds)
    best_labels = bag_round_stop[rounds]

    # Determine the best journey to all possible destination stations
    journey_to_destinations = dict()
    for destination_station_name, to_stops in destination_stops.items():
        dest_stop = best_stop_at_target_station(to_stops, best_labels)
        if dest_stop != 0:
            journey = reconstruct_journey(dest_stop, best_labels)
            journey_to_destinations[destination_station_name] = journey

    return journey_to_destinations

In [44]:
# if __name__ == "__main__":
#     args = parse_arguments()
#     main(
#         args.input,
#         args.origin,
#         args.destination,
#         args.time,
#         args.rounds,
#     )

### Arguments

In [45]:
input_folder = "data/output"
origin_station = "Arnhem Zuid"
destination_station = "Oosterbeek"
departure_time = "08:30:00"
rounds = 5

### query_raptor

To query raptor, we must run main()

In [46]:
main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
)

2022-06-10 16:35:57.742 | DEBUG    | __main__:main:10 - Input directory     : data/output
2022-06-10 16:35:57.742 | DEBUG    | __main__:main:11 - Origin station      : Arnhem Zuid
2022-06-10 16:35:57.743 | DEBUG    | __main__:main:12 - Destination station : Oosterbeek
2022-06-10 16:35:57.744 | DEBUG    | __main__:main:13 - Departure time      : 08:30:00
2022-06-10 16:35:57.745 | DEBUG    | __main__:main:14 - Rounds              : 5
2022-06-10 16:35:57.746 | INFO     | __main__:main:18 - Calculating network from: Arnhem Zuid
2022-06-10 16:35:57.747 | DEBUG    | __main__:main:22 - Departure time (s.)  : 30600
2022-06-10 16:35:57.751 | DEBUG    | __main__:run:24 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-06-10 16:35:57.752 | INFO     | __main__:run:36 - Analyzing possibilities round 1
2022-06-10 16:35:57.767 | DEBUG    | __main__:run:40 - Stops to evaluate count: 2
2022-06-10 16:35:57.768 | DEBUG    | __main__:traverse_routes:105 - Traver

### Step-by-step

In [47]:
"""Run RAPTOR algorithm"""

logger.debug("Input directory     : {}", input_folder)
logger.debug("Origin station      : {}", origin_station)
logger.debug("Destination station : {}", destination_station)
logger.debug("Departure time      : {}", departure_time)
logger.debug("Rounds              : {}", str(rounds))

# timetable = read_timetable(input_folder)

logger.info(f"Calculating network from: {origin_station}")

# Departure time seconds
dep_secs = str2sec(departure_time)
logger.debug("Departure time (s.)  : " + str(dep_secs))

2022-06-10 16:35:59.094 | DEBUG    | __main__:<cell line: 3>:3 - Input directory     : data/output
2022-06-10 16:35:59.095 | DEBUG    | __main__:<cell line: 4>:4 - Origin station      : Arnhem Zuid
2022-06-10 16:35:59.095 | DEBUG    | __main__:<cell line: 5>:5 - Destination station : Oosterbeek
2022-06-10 16:35:59.096 | DEBUG    | __main__:<cell line: 6>:6 - Departure time      : 08:30:00
2022-06-10 16:35:59.096 | DEBUG    | __main__:<cell line: 7>:7 - Rounds              : 5
2022-06-10 16:35:59.096 | INFO     | __main__:<cell line: 11>:11 - Calculating network from: Arnhem Zuid
2022-06-10 16:35:59.097 | DEBUG    | __main__:<cell line: 15>:15 - Departure time (s.)  : 30600


#### run_raptor

In [48]:
### run_raptor()

"""
# Find route between two stations
journey_to_destinations = run_raptor(
    timetable,
    origin_station,
    dep_secs,
    rounds,
)

# Print journey to destination
journey_to_destinations[destination_station].print(dep_secs=dep_secs)
"""


# Get stops for origin and all destinations
from_stops = timetable.stations.get(origin_station).stops
logger.debug(f"from_stops                   : {from_stops}")
"""When RAPTOR is satisfying a shortest-path query,
possible destinations include all stops other than origin""" 
destination_stops = {
    st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
}
destination_stops.pop(origin_station, None) # Remove origin_station from the possible destination_stops
assert len(destination_stops) == len(timetable.stations) - 1
logger.debug(f"number of destination_stops  : {len(destination_stops)}")

# Run Round-Based Algorithm
raptor = RaptorAlgorithm(timetable)
# bag_round_stop = raptor.run(from_stops, dep_secs, rounds)
# best_labels = bag_round_stop[rounds]

# # Determine the best journey to all possible destination stations
# journey_to_destinations = dict()
# for destination_station_name, to_stops in destination_stops.items():
#     dest_stop = best_stop_at_target_station(to_stops, best_labels)
#     if dest_stop != 0:
#         journey = reconstruct_journey(dest_stop, best_labels)
#         journey_to_destinations[destination_station_name] = journey

# return journey_to_destinations

2022-06-10 16:35:59.222 | DEBUG    | __main__:<cell line: 19>:19 - from_stops                   : [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-06-10 16:35:59.223 | DEBUG    | __main__:<cell line: 27>:27 - number of destination_stops  : 247


Next in code:

```python
bag_round_stop = raptor.run(from_stops, dep_secs, rounds)
```

<br>

- We break down `RaptorAlgorithm` step-by-step below:

#### accumulate_routes

In [49]:
### Implementation of `bag_round_stop = raptor.run(from_stops, dep_secs, rounds)`

# def accumulate_routes(self, marked_stops: List[Stop]) -> List[Tuple[Route, Stop]]:
def accumulate_routes(marked_stops: List[Stop]) -> List[Tuple[Route, Stop]]:
    """Accumulate routes serving marked stops from previous round, i.e. Q"""
    route_marked_stops = {}  # i.e. Q
    for marked_stop in marked_stops:
        routes_serving_stop = self_timetable.routes.get_routes_of_stop(marked_stop)
        for route in routes_serving_stop:
            # Check if new_stop is before existing stop in Q

            # Get value for key (route), return None if key not in dictionary
            current_stop_for_route = route_marked_stops.get(route, None)  # p'
            # If we haven't marked this route stop
            # or
            # if current_stop_for_route comes after the marked stop???
            if (current_stop_for_route is None) or (
                route.stop_index(current_stop_for_route)
                > route.stop_index(marked_stop)
            ):
                route_marked_stops[route] = marked_stop
    route_marked_stops = [(r, p) for r, p in route_marked_stops.items()]

    return route_marked_stops

#### traverse_routes

In [50]:
### Implementation of `bag_round_stop = raptor.run(from_stops, dep_secs, rounds)`

def traverse_routes(
    # self,
    bag_round_stop: Dict[int, Dict[Stop, Label]],
    k: int,
    route_marked_stops: List[Tuple[Route, Stop]],
) -> Tuple:
    """
    Iterator through the stops reachable and add all new reachable stops
    by following all trips from the reached stations. Trips are only followed
    in the direction of travel and beyond already added points.

    :param bag_round_stop: Bag per round per stop
    :param k: current round
    :param route_marked_stops: list of marked (route, stop) for evaluation
    """
    logger.debug(f"Traverse routes for round {k}")

    bag_round_stop = deepcopy(bag_round_stop)
    new_stops = []
    n_evaluations = 0
    n_improvements = 0

    # For each route
    for (marked_route, marked_stop) in route_marked_stops:

        # Current trip for this marked stop
        current_trip = None

        # Iterate over all stops after current stop within the current route
        current_stop_index = marked_route.stop_index(marked_stop)
        remaining_stops_in_route = marked_route.stops[current_stop_index:]
        boarding_stop = None

        for current_stop_index, current_stop in enumerate(remaining_stops_in_route):
            # Can the label be improved in this round?
            n_evaluations += 1

            # t != _|_
            if current_trip is not None:
                # Arrival time at stop, i.e. arr(current_trip, next_stop)
                new_arrival_time = current_trip.get_stop(current_stop).dts_arr
                best_arrival_time = self_bag_star[
                    current_stop
                ].earliest_arrival_time

                if new_arrival_time < best_arrival_time:
                    # Update arrival by trip, i.e.
                    #   t_k(next_stop) = t_arr(t, pi)
                    #   t_star(p_i) = t_arr(t, pi)

                    bag_round_stop[k][current_stop].update(
                        new_arrival_time, current_trip, boarding_stop
                    )
                    self_bag_star[current_stop].update(
                        new_arrival_time, current_trip, boarding_stop
                    )

                    # Logging
                    n_improvements += 1
                    new_stops.append(current_stop)

            # Can we catch an earlier trip at p_i
            # if tau_{k-1}(next_stop) <= tau_dep(t, next_stop)
            previous_earliest_arrival_time = bag_round_stop[k][
                current_stop
            ].earliest_arrival_time
            earliest_trip_stop_time = marked_route.earliest_trip_stop_time(
                previous_earliest_arrival_time, current_stop
            )
            if (
                earliest_trip_stop_time is not None
                and previous_earliest_arrival_time
                <= earliest_trip_stop_time.dts_dep
            ):
                current_trip = earliest_trip_stop_time.trip
                boarding_stop = current_stop

    logger.debug(f"- Evaluations    : {n_evaluations}")
    logger.debug(f"- Improvements   : {n_improvements}")

    return bag_round_stop, new_stops

#### get_transfer_time

In [51]:
### Implementation of `bag_round_stop = raptor.run(from_stops, dep_secs, rounds)`

# def get_transfer_time(self, stop_from: Stop, stop_to: Stop) -> int:
def get_transfer_time(stop_from: Stop, stop_to: Stop) -> int:
    """
    Calculate the transfer time from a stop to another stop (usually at one station)
    """
    transfers = self_timetable.transfers
    return transfers.stop_to_stop_idx[(stop_from, stop_to)].layovertime

#### add_transfer_time

In [52]:
### Implementation of `bag_round_stop = raptor.run(from_stops, dep_secs, rounds)`

def add_transfer_time(
    # self,
    bag_round_stop: Dict[int, Dict[Stop, Label]],
    k: int,
    marked_stops: List[Stop],
) -> Tuple:
    """
    Add transfers between platforms.

    :param bag_round_stop: Label per round per stop
    :param k: current round
    :param marked_stops: list of marked stops for evaluation
    """

    new_stops = []

    # Add in transfers to other platforms
    for current_stop in marked_stops:
        other_station_stops = [
            st for st in current_stop.station.stops if st != current_stop
        ]

        time_sofar = bag_round_stop[k][current_stop].earliest_arrival_time
        for arrive_stop in other_station_stops:
            new_earliest_arrival = time_sofar + get_transfer_time(
                current_stop, arrive_stop
            )
            previous_earliest_arrival = self_bag_star[
                arrive_stop
            ].earliest_arrival_time

            # Domination criteria
            if new_earliest_arrival < previous_earliest_arrival:
                bag_round_stop[k][arrive_stop].update(
                    new_earliest_arrival,
                    TRANSFER_TRIP,
                    current_stop,
                )
                self_bag_star[arrive_stop].update(
                    new_earliest_arrival, TRANSFER_TRIP, current_stop
                )
                new_stops.append(arrive_stop)

    return bag_round_stop, new_stops

#### RaptorAlgorithm

In [53]:
### Implementation of `bag_round_stop = raptor.run(from_stops, dep_secs, rounds)`

# class RaptorAlgorithm:
"""RAPTOR Algorithm"""

# def __init__(self, timetable: Timetable):
self_timetable = timetable
self_bag_star = None

# def run(self, from_stops, dep_secs, rounds) -> Dict[int, Dict[Stop, Label]]:
"""Run Round-Based Algorithm"""

# Initialize empty bag of labels, i.e. B_k(p) = Label() for every k and p
bag_round_stop: Dict[int, Dict[Stop, Label]] = {}
for k in range(0, rounds + 1):
    bag_round_stop[k] = {}
    for p in self_timetable.stops:
        bag_round_stop[k][p] = Label()

# Initialize bag with earliest arrival tiems
self_bag_star = {}
for p in self_timetable.stops:
    self_bag_star[p] = Label()

# Initialize bag with start node taking DEP_SECS seconds to reach
logger.debug(f"Starting from Stop IDs: {str(from_stops)}")
marked_stops = []
for from_stop in from_stops:
    # Update Label with departure time
    # bag_round_stop[0][from_stop].update(earliest_arrival_time=dep_secs, fare_addition=None, from_stop=None)
    bag_round_stop[0][from_stop].update(dep_secs, None, None)
    # self_bag_star[from_stop].update(earliest_arrival_time=dep_secs, fare_addition=None, from_stop=None)
    self_bag_star[from_stop].update(dep_secs, None, None)
    marked_stops.append(from_stop)

# Run rounds
for k in range(1, rounds + 1):
    logger.info(f"Analyzing possibilities round {k}")
    bag_round_stop[k] = deepcopy(bag_round_stop[k - 1])

    # Get list of stops to evaluate in the process
    logger.debug(f"Stops to evaluate count: {len(marked_stops)}")

    # Get marked route stops - A list of routes serving marked stops from previous round, i.e. Q
    route_marked_stops = accumulate_routes(marked_stops)



    ### HERE ###########################################################



    # Update time to stops calculated based on stops reachable
    bag_round_stop, marked_trip_stops = traverse_routes(
        bag_round_stop, k, route_marked_stops
    )
    logger.debug(f"{len(marked_trip_stops)} reachable stops added")

    # Add footpath transfers and update
    bag_round_stop, marked_transfer_stops = add_transfer_time(
        bag_round_stop, k, marked_trip_stops
    )
    logger.debug(f"{len(marked_transfer_stops)} transferable stops added")

    marked_stops = set(marked_trip_stops).union(marked_transfer_stops)
    logger.debug(f"{len(marked_stops)} stops to evaluate in next round")

# return bag_round_stop

2022-06-10 16:35:59.983 | DEBUG    | __main__:<cell line: 26>:26 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-06-10 16:35:59.984 | INFO     | __main__:<cell line: 37>:38 - Analyzing possibilities round 1
2022-06-10 16:35:59.998 | DEBUG    | __main__:<cell line: 37>:42 - Stops to evaluate count: 2
2022-06-10 16:35:59.999 | DEBUG    | __main__:traverse_routes:18 - Traverse routes for round 1
2022-06-10 16:36:00.159 | DEBUG    | __main__:traverse_routes:80 - - Evaluations    : 293
2022-06-10 16:36:00.159 | DEBUG    | __main__:traverse_routes:81 - - Improvements   : 64
2022-06-10 16:36:00.160 | DEBUG    | __main__:<cell line: 37>:57 - 64 reachable stops added
2022-06-10 16:36:00.161 | DEBUG    | __main__:<cell line: 37>:63 - 78 transferable stops added
2022-06-10 16:36:00.161 | DEBUG    | __main__:<cell line: 37>:66 - 92 stops to evaluate in next round
2022-06-10 16:36:00.161 | INFO     | __main__:<cell line: 37>:38 - Analyzing possibilities

In [54]:
# bag_round_stop = raptor.run(from_stops, dep_secs, rounds)
best_labels = bag_round_stop[rounds]

[Paper (Section 2, Paragraph 2):](raptor_alenex.pdf)

"More precisely, the algorithm associates with each stop $p$ a multilabel ($\tau_{0}(p)$, $\tau_{1}(p)$), ..., $\tau_{K}(p)$), where $\tau_{i}(p)$ represents the earliest known arrival time at $p$ with up to $i$ trips."

<br>

`bag_round_stop` = multilabel ($\tau_{0}(p)$, $\tau_{1}(p)$), ..., $\tau_{K}(p)$)

`best_labels` = $\tau_{K}(p)$

In [55]:
# The best labels (for reaching other stops), after ROUND 5
rounds, len(best_labels), best_labels[next(iter(best_labels))]

(5,
 736,
 Label(earliest_arrival_time=41940, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Vlissingen Souburg-1 [2423728])))

In [56]:
# Determine the best journey to all possible destination stations
journey_to_destinations = dict()
for destination_station_name, to_stops in destination_stops.items():
    # At a given station, find the destination Stop with the shortest distance (e.g. earliest arrival time).
    # Required in order to prevent adding travel time to the arrival time.
    dest_stop = best_stop_at_target_station(to_stops, best_labels)
    if dest_stop != 0:
        journey = reconstruct_journey(dest_stop, best_labels)
        journey_to_destinations[destination_station_name] = journey

#### journey

In [57]:
### main()

# Print journey to destination
journey_to_destinations[destination_station].print(dep_secs=dep_secs)

2022-06-10 16:36:01.706 | INFO     | pyraptor.model.structures:print:1034 - Journey:
2022-06-10 16:36:01.707 | INFO     | pyraptor.model.structures:print:1057 - 08:44 Arnhem Zuid         (p.   2) TO 08:49 Arnhem Centraal     (p.   3) WITH 7625
2022-06-10 16:36:01.708 | INFO     | pyraptor.model.structures:print:1057 - 09:05 Arnhem Centraal     (p.  10) TO 09:09 Oosterbeek          (p.   1) WITH 7528
2022-06-10 16:36:01.708 | INFO     | pyraptor.model.structures:print:1059 - Fare: €0
2022-06-10 16:36:01.709 | INFO     | pyraptor.model.structures:print:1060 - Occupancy: 0.0
2022-06-10 16:36:01.709 | INFO     | pyraptor.model.structures:print:1067 - Duration: 00:25 (00:39 from request time 08:30)
2022-06-10 16:36:01.710 | INFO     | pyraptor.model.structures:print:1068 - 


<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

---

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

# McRaptor

- Task: Duplicate the McRaptorAlgorithm found in [pyraptor/model/mcraptor.py](pyraptor/model/mcraptor.py)
- Purpose: Reference, implementation of occupancy

## Imports

In [58]:
from typing import List, Tuple, Dict
from copy import copy
from time import perf_counter

from loguru import logger
from pyraptor.model.structures import (
    Timetable,
    Stop,
    Route,
    Bag,
    Label,
    Leg,
    Journey,
    pareto_set,
)

## McRaptorAlgorithm

In [59]:
class McRaptorAlgorithm:
    """McRAPTOR Algorithm"""

    def __init__(self, timetable: Timetable):
        self.timetable = timetable

    def run(
        self, from_stops: List[Stop], dep_secs: int, rounds: int, previous_run: Dict[int, Bag] = None
    ) -> Dict[int, Dict[int, Bag]]:
        """Run Round-Based Algorithm"""

        s = perf_counter()

        # Initialize empty bag, i.e. B_k(p) = [] for every k and p
            # The bag, B_k(p), stores non-dominating labels, L, for each stop, p, in round k
                # Is this the id of the trip which dominates? No.
                    # It is a class with multiple attributes,
                    # E.g. earliest_arrival_time, fare, n_trips (defined in criteria)
        bag_round_stop: Dict[int, Dict[Stop, Bag]] = {}
        for k in range(0, rounds + 1):
            bag_round_stop[k] = {}
            for p in self.timetable.stops:
                bag_round_stop[k][p] = Bag()

        # Add origin stops to bag
        logger.debug(f"Starting from Stop IDs: {str(from_stops)}")

        # Initialize bag for round 0, i.e. add Labels with criterion 0 for all from stops
        if previous_run != None:
            # For the range query
            bag_round_stop[0] = copy(previous_run)

        for from_stop in from_stops:
            # bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop))
            bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop, 0))

        marked_stops = from_stops

        # Run rounds
        actual_rounds = 0
        for k in range(1, rounds + 1):
            logger.info(f"Analyzing possibilities round {k}")
            logger.debug(f"Stops to evaluate count: {len(marked_stops)}")

            # Copy bag from previous round
            bag_round_stop[k] = copy(bag_round_stop[k - 1])

            if len(marked_stops) > 0:
                actual_rounds = k

                # Accumulate routes serving marked stops from previous round
                route_marked_stops = self.accumulate_routes(marked_stops)

                # Traverse each route
                bag_round_stop, marked_stops_trips = self.traverse_route(
                    bag_round_stop, k, route_marked_stops
                )

                # Now add footpath transfers and update
                bag_round_stop, marked_stops_transfers = self.add_transfer_time(
                    bag_round_stop, k, marked_stops_trips
                )

                marked_stops = set(marked_stops_trips).union(marked_stops_transfers)
            else:
                break

        logger.info("Finish round-based algorithm to create bag with best labels")
        logger.info(f"Running time: {perf_counter() - s}")

        return bag_round_stop, actual_rounds

    def accumulate_routes(self, marked_stops) -> List[Tuple[Route, Stop]]:
        """Accumulate routes serving marked stops from previous round, i.e. Q"""
        route_marked_stops = {}  # i.e. Q
        for marked_stop in marked_stops:
            routes_serving_stop = self.timetable.routes.get_routes_of_stop(marked_stop)
            for route in routes_serving_stop:
                # Check if new_stop is before existing stop in Q
                current_stop_for_route = route_marked_stops.get(route, None)  # p'
                if (current_stop_for_route is None) or (
                    route.stop_index(current_stop_for_route)
                    > route.stop_index(marked_stop)
                ):
                    route_marked_stops[route] = marked_stop
        route_marked_stops = [(r, p) for r, p in route_marked_stops.items()]

        logger.debug(f"Found {len(route_marked_stops)} routes serving marked stops")

        return route_marked_stops

    def traverse_route(
        self,
        bag_round_stop: Dict[int, Dict[int, Bag]],
        k: int,
        route_marked_stops: List[Tuple[Route, Stop]],
    ) -> Tuple[Dict[int, Dict[int, Bag]], List[Stop]]:
        """
        Traverse through all marked route-stops and update labels accordingly.

        :param bag_round_stop: Bag per round per stop
        :param k: current round
        :param route_marked_stops: list of marked (route, stop) for evaluation

        OCCUPANCY:
        To update occupancy for this label, we will need access to previous trips/legs.
        """

        new_marked_stops = set()

        for (marked_route, marked_stop) in route_marked_stops:
            # Traversing through route from marked stop
            route_bag = Bag()

            # Get all stops after current stop within the current route
            marked_stop_index = marked_route.stop_index(marked_stop)
            remaining_stops_in_route = marked_route.stops[marked_stop_index:]

            # For the rest of the trips (stops) on this route
                # Add the occupancy for this trip
            for stop_idx, current_stop in enumerate(remaining_stops_in_route):

                # Step 1: update earliest arrival times and criteria for each label L in route-bag
                update_labels = []
                for label in route_bag.labels:
                    trip_stop_time = label.trip.get_stop(current_stop)

                    # Take fare of previous stop in trip as fare is defined on start
                    # Take OCCUPANCY from previous stop in trip as occupancy defined on start
                    previous_stop = remaining_stops_in_route[stop_idx - 1]
                    from_fare = label.trip.get_fare(previous_stop)
                    from_occupancy = label.trip.get_stop_occupancy(previous_stop) # 06/06/2022

                    label = label.update(
                        earliest_arrival_time=trip_stop_time.dts_arr,
                        fare_addition=from_fare,
                        occupancy_addition=from_occupancy,
                    )

                    update_labels.append(label)
                route_bag = Bag(labels=update_labels)

                # Step 2: merge bag_route into bag_round_stop and remove dominated labels
                # The label contains the trip with which one arrives at current stop with k legs
                # and we boarded the trip at from_stop.
                bag_round_stop[k][current_stop] = bag_round_stop[k][current_stop].merge(
                    route_bag
                )
                bag_update = bag_round_stop[k][current_stop].update

                # Mark stop if bag is updated
                if bag_update:
                    new_marked_stops.add(current_stop)

                # Step 3: merge B_{k-1}(p) into B_r
                route_bag = route_bag.merge(bag_round_stop[k - 1][current_stop])

                # Assign trips to all newly added labels in route_bag
                # This is the trip on which we board
                update_labels = []
                for label in route_bag.labels:
                    earliest_trip = marked_route.earliest_trip(
                        label.earliest_arrival_time, current_stop
                    )
                    if earliest_trip is not None:
                        # Update label with earliest trip in route leaving from this station
                        # If trip is different we board the trip at current_stop
                        label = label.update_trip(earliest_trip, current_stop)
                        update_labels.append(label)
                route_bag = Bag(labels=update_labels)

        logger.debug(f"{len(new_marked_stops)} reachable stops added")

        return bag_round_stop, new_marked_stops

    def add_transfer_time(
        self,
        bag_round_stop: Dict[int, Dict[Stop, Bag]],
        k: int,
        marked_stops: List[Stop],
    ) -> Tuple:
        """Add transfers between platforms."""

        marked_stops_transfers = set()

        # Add in transfers to other platforms
        for stop in marked_stops:
            other_station_stops = [st for st in stop.station.stops if st != stop]

            for other_stop in other_station_stops:
                # Create temp copy of B_k(p_i)
                temp_bag = Bag()
                for label in bag_round_stop[k][stop].labels:
                    # Add arrival time to each label
                    transfer_arrival_time = (
                        label.earliest_arrival_time
                        + self.get_transfer_time(stop, other_stop)
                    )
                    # Update label with new earliest arrival time at other_stop
                    label = label.update(
                        earliest_arrival_time=transfer_arrival_time,
                        fare_addition=0,
                        from_stop=stop,
                    )
                    temp_bag.add(label)

                # Merg temp bag into B_k(p_j)
                bag_round_stop[k][other_stop] = bag_round_stop[k][other_stop].merge(
                    temp_bag
                )
                bag_update = bag_round_stop[k][other_stop].update

                # Mark stop if bag is updated
                if bag_update:
                    marked_stops_transfers.add(other_stop)

        logger.debug(f"{len(marked_stops_transfers)} transferable stops added")

        return bag_round_stop, marked_stops_transfers

    def get_transfer_time(self, stop_from: Stop, stop_to: Stop) -> int:
        """
        Calculate the transfer time from a stop to another stop (usually at one station)
        """
        transfers = self.timetable.transfers
        return transfers.stop_to_stop_idx[(stop_from, stop_to)].layovertime


## best_legs_to_destination_station

In [60]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

def best_legs_to_destination_station(
    to_stops: List[Stop], last_round_bag: Dict[Stop, Bag]
) -> List[Leg]:
    """
    Find the last legs to destination station that are reached by non-dominated labels.
    """

    # Find all labels to target_stops
    best_labels = [
        (stop, label) for stop in to_stops for label in last_round_bag[stop].labels
    ]

    # TODO Use merge function on Bag
    # Pareto optimal labels
    pareto_optimal_labels = pareto_set([label for (_, label) in best_labels])
    pareto_optimal_labels = [
        (stop, label) for (stop, label) in best_labels if label in pareto_optimal_labels
    ]

    # Label to leg, i.e. add to_stop
    legs = [
        Leg(
            label.from_stop, # from_stop: Stop
            to_stop, # to_stop: Stop
            label.trip, # trip: Trip
            label.earliest_arrival_time, # earliest_arrival_time: int
            label.fare, # fare: int = 0
            # label.occupancy, # occupancy: int = 0 # 03/06/2022
            label.n_trips, # n_trips: int = 0
        )
        for (to_stop, label) in pareto_optimal_labels
    ]
    return legs

## query_mcraptor

### Imports

In [61]:
"""Run query with RAPTOR algorithm"""
import argparse
from typing import List, Dict
from copy import copy
from time import perf_counter

from loguru import logger

from pyraptor.dao.timetable import read_timetable
from pyraptor.model.structures import Timetable, Journey, Station
# from pyraptor.model.mcraptor import (
#     McRaptorAlgorithm,
#     reconstruct_journeys,
#     best_legs_to_destination_station,
# )
from pyraptor.util import str2sec

### main

In [62]:
def main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
):
    """Run RAPTOR algorithm"""

    logger.debug("Input directory     : {}", input_folder)
    logger.debug("Origin station      : {}", origin_station)
    logger.debug("Destination station : {}", destination_station)
    logger.debug("Departure time      : {}", departure_time)
    logger.debug("Rounds              : {}", str(rounds))

    # timetable = read_timetable(input_folder)

    logger.info(f"Calculating network from : {origin_station}")

    # Departure time seconds
    dep_secs = str2sec(departure_time)
    logger.debug("Departure time (s.)  : " + str(dep_secs))

    # Find route between two stations
    journeys_to_destinations = run_mcraptor(
        timetable,
        origin_station,
        dep_secs,
        rounds,
    )

    # Output journey
    journeys = journeys_to_destinations[destination_station]
    if len(journeys) != 0:
        for jrny in journeys:
            jrny.print(dep_secs=dep_secs)

### reconstruct_journeys

In [63]:
def reconstruct_journeys(
    from_stops: List[Stop],
    destination_legs: List[Leg],
    bag_round_stop: Dict[int, Dict[Stop, Bag]],
    k: int,
) -> List[Journey]:
    """
    Construct Journeys for destinations from bags by recursively
    looping from destination to origin.
    """

    def loop(
        bag_round_stop: Dict[int, Dict[Stop, Bag]], k: int, journeys: List[Journey]
    ):
        """Create full journey by prepending legs recursively"""

        last_round_bags = bag_round_stop[k]

        for jrny in journeys:
            current_leg = jrny[0]

            # End of journey if we are at origin stop or journey is not feasible
            if current_leg.trip is None or current_leg.from_stop in from_stops:
                jrny = jrny.remove_transfer_legs()
                if jrny.is_valid() is True:
                    yield jrny
                continue

            ###################################################################

            # HERE - Update occupancy

            ###################################################################
            # Loop trough each new leg
            labels_to_from_stop = last_round_bags[current_leg.from_stop].labels
            for new_label in labels_to_from_stop:

                """
                If we call the new_label.update() method here,
                we would be updating the Label, not the TripStopTime.
                Hence, it would be reset every time we do a new search.

                Instead, we need to update the TripStopTime occupancy.
                We do this by getting the TripStopTime instance by calling the
                trip.get_stop method and calling our custom update_occupancy
                method on the resulting TripStopTime instance.

                CHECK:
                We should only instanctiate occupancy as 0 ONCE,
                during the initialisation of the TripStopTime instance.
                Thereonin, we should intitialise occupancy to None,
                and/or inherit from the TripStopTime occupancy.
                """

                # new_label.trip.get_stop_occupancy()
                # print(f"new_label.trip: {new_label.trip}")
                # new_label.trip.get_stop(current_leg.from_stop).update_occupancy()
                # new_label.trip.update_occupancy(current_leg.from_stop) # 08/06/2022

                """We don't want to update occupancy here, this does it for all journeys from origin.
                We only want to do so between the chosen journey"""

                new_leg = Leg(
                    new_label.from_stop, # from_stop: Stop
                    current_leg.from_stop, # to_stop: Stop
                    new_label.trip, # trip: Trip
                    new_label.earliest_arrival_time, # earliest_arrival_time: int
                    new_label.fare, # fare: int = 0
                    # new_label.occupancy, # occupancy: int = 0 # 03/06/2022
                    new_label.n_trips, # n_trips: int = 0
                )
                # Only add new_leg if compatible before current leg, e.g. earlier arrival time, etc.
                if new_leg.is_compatible_before(current_leg):
                    new_jrny = jrny.prepend_leg(new_leg)
                    for i in loop(bag_round_stop, k, [new_jrny]):
                        yield i

    journeys = [Journey(legs=[leg]) for leg in destination_legs]
    journeys = [jrny for jrny in loop(bag_round_stop, k, journeys)]

    return journeys

### run_mcraptor

In [64]:
def run_mcraptor(
    timetable: Timetable,
    origin_station: str,
    dep_secs: int,
    rounds: int,
) -> Dict[Station, List[Journey]]:
    """
    Perform the McRaptor algorithm.

    :param timetable: timetable
    :param origin_station: Name of origin station
    :param dep_secs: Time of departure in seconds
    :param rounds: Number of iterations to perform
    """

    # Run Round-Based Algorithm for an origin station
    from_stops = timetable.stations.get(origin_station).stops
    raptor = McRaptorAlgorithm(timetable)
    bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)
    last_round_bag = copy(bag_round_stop[rounds])

    # Calculate journets to all destinations
    logger.info("Calculating journeys to all destinations")
    s = perf_counter()

    destination_stops = {
        st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
    }
    destination_stops.pop(origin_station, None)

    journeys_to_destinations = dict()
    for destination_station_name, to_stops in destination_stops.items():
        destination_legs = best_legs_to_destination_station(to_stops, last_round_bag)

        if len(destination_legs) == 0:
            logger.info("Destination unreachable with given parameters")
            continue

        journeys = reconstruct_journeys(
            from_stops, destination_legs, bag_round_stop, k=rounds
        )
        journeys_to_destinations[destination_station_name] = journeys

    logger.info(f"Journey calculation time: {perf_counter() - s}")

    return journeys_to_destinations

In [65]:
# if __name__ == "__main__":
#     args = parse_arguments()
#     main(
#         args.input,
#         args.origin,
#         args.destination,
#         args.time,
#         args.rounds,
#     )

### Arguments

In [66]:
input_folder = "data/output"
origin_station = "Arnhem Zuid"
destination_station = "Oosterbeek"
departure_time = "08:30:00"
rounds = 5

### query_raptor

To query raptor, we must run main()

In [67]:
main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
)

2022-06-10 16:36:03.118 | DEBUG    | __main__:main:10 - Input directory     : data/output
2022-06-10 16:36:03.119 | DEBUG    | __main__:main:11 - Origin station      : Arnhem Zuid
2022-06-10 16:36:03.119 | DEBUG    | __main__:main:12 - Destination station : Oosterbeek
2022-06-10 16:36:03.120 | DEBUG    | __main__:main:13 - Departure time      : 08:30:00
2022-06-10 16:36:03.120 | DEBUG    | __main__:main:14 - Rounds              : 5
2022-06-10 16:36:03.121 | INFO     | __main__:main:18 - Calculating network from : Arnhem Zuid
2022-06-10 16:36:03.121 | DEBUG    | __main__:main:22 - Departure time (s.)  : 30600
2022-06-10 16:36:03.126 | DEBUG    | __main__:run:26 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-06-10 16:36:03.126 | INFO     | __main__:run:42 - Analyzing possibilities round 1
2022-06-10 16:36:03.126 | DEBUG    | __main__:run:43 - Stops to evaluate count: 2
2022-06-10 16:36:03.127 | DEBUG    | __main__:accumulate_routes:88 - Foun

### Step-by-step

In [101]:
"""Run RAPTOR algorithm"""

logger.debug("Input directory     : {}", input_folder)
logger.debug("Origin station      : {}", origin_station)
logger.debug("Destination station : {}", destination_station)
logger.debug("Departure time      : {}", departure_time)
logger.debug("Rounds              : {}", str(rounds))

# timetable = read_timetable(input_folder)

logger.info(f"Calculating network from : {origin_station}")

# Departure time seconds
dep_secs = str2sec(departure_time)
logger.debug("Departure time (s.)  : " + str(dep_secs))

2022-06-10 16:39:04.743 | DEBUG    | __main__:<cell line: 3>:3 - Input directory     : data/output
2022-06-10 16:39:04.752 | DEBUG    | __main__:<cell line: 4>:4 - Origin station      : Arnhem Zuid
2022-06-10 16:39:04.756 | DEBUG    | __main__:<cell line: 5>:5 - Destination station : Oosterbeek
2022-06-10 16:39:04.759 | DEBUG    | __main__:<cell line: 6>:6 - Departure time      : 08:30:00
2022-06-10 16:39:04.760 | DEBUG    | __main__:<cell line: 7>:7 - Rounds              : 5
2022-06-10 16:39:04.761 | INFO     | __main__:<cell line: 11>:11 - Calculating network from : Arnhem Zuid
2022-06-10 16:39:04.762 | DEBUG    | __main__:<cell line: 15>:15 - Departure time (s.)  : 30600


In [102]:
timetable.trip_stop_times.stop_trip_idx[Stop_eg][0].occupancy

2.0

#### run_mcraptor

In [103]:
"""
# Find route between two stations
journeys_to_destinations = run_mcraptor(
    timetable,
    origin_station,
    dep_secs,
    rounds,
)

# Output journey
journeys = journeys_to_destinations[destination_station]
if len(journeys) != 0:
    for jrny in journeys:
        jrny.print(dep_secs=dep_secs)
"""

# def run_mcraptor(
# timetable: Timetable,
# origin_station: str,
# dep_secs: int,
# rounds: int,
# ) -> Dict[Station, List[Journey]]:
# """
# Perform the McRaptor algorithm.

# :param timetable: timetable
# :param origin_station: Name of origin station
# :param dep_secs: Time of departure in seconds
# :param rounds: Number of iterations to perform
# """

# Run Round-Based Algorithm for an origin station
from_stops = timetable.stations.get(origin_station).stops
raptor = McRaptorAlgorithm(timetable)
# bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)
# last_round_bag = copy(bag_round_stop[rounds])

# # Calculate journets to all destinations
# logger.info("Calculating journeys to all destinations")
# s = perf_counter()

# destination_stops = {
#     st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
# }
# destination_stops.pop(origin_station, None)

# journeys_to_destinations = dict()
# for destination_station_name, to_stops in destination_stops.items():
#     destination_legs = best_legs_to_destination_station(to_stops, last_round_bag)

#     if len(destination_legs) == 0:
#         logger.info("Destination unreachable with given parameters")
#         continue

#     journeys = reconstruct_journeys(
#         from_stops, destination_legs, bag_round_stop, k=rounds
#     )
#     journeys_to_destinations[destination_station_name] = journeys

# logger.info(f"Journey calculation time: {perf_counter() - s}")

# return journeys_to_destinations

Next in code:

```python
bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)
```

<br>

- We break down `McRaptorAlgorithm` step-by-step below:

In [104]:
# def __init__(self, timetable: Timetable):
self_timetable = timetable

In [105]:
previous_run: Dict[int, Bag] = None

# def run(
#     self, from_stops: List[Stop], dep_secs: int, rounds: int, previous_run: Dict[int, Bag] = None
# ) -> Dict[int, Dict[int, Bag]]:
#     """Run Round-Based Algorithm"""

s = perf_counter()

# Initialize empty bag, i.e. B_k(p) = [] for every k and p
    # The bag, B_k(p), stores non-dominating labels, L, for each stop, p, in round k
        # Is this the id of the trip which dominates? No.
            # It is a class with multiple attributes,
            # E.g. earliest_arrival_time, fare, n_trips (defined in criteria)
bag_round_stop: Dict[int, Dict[Stop, Bag]] = {}
for k in range(0, rounds + 1):
    bag_round_stop[k] = {}
    for p in self_timetable.stops:
        bag_round_stop[k][p] = Bag()

# Add origin stops to bag
logger.debug(f"Starting from Stop IDs: {str(from_stops)}")

# Initialize bag for round 0, i.e. add Labels with criterion 0 for all from stops
if previous_run != None:
    # For the range query
    bag_round_stop[0] = copy(previous_run)

for from_stop in from_stops:
    # bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop))
    bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop, 0))

marked_stops = from_stops

2022-06-10 16:39:07.976 | DEBUG    | __main__:<cell line: 22>:22 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]


#### accumulate_routes

In [106]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

# def accumulate_routes(self, marked_stops) -> List[Tuple[Route, Stop]]:
def accumulate_routes(marked_stops) -> List[Tuple[Route, Stop]]:
    """Accumulate routes serving marked stops from previous round, i.e. Q"""
    route_marked_stops = {}  # i.e. Q
    for marked_stop in marked_stops:
        routes_serving_stop = self_timetable.routes.get_routes_of_stop(marked_stop)
        for route in routes_serving_stop:
            # Check if new_stop is before existing stop in Q
            current_stop_for_route = route_marked_stops.get(route, None)  # p'
            if (current_stop_for_route is None) or (
                route.stop_index(current_stop_for_route)
                > route.stop_index(marked_stop)
            ):
                route_marked_stops[route] = marked_stop
    route_marked_stops = [(r, p) for r, p in route_marked_stops.items()]

    logger.debug(f"Found {len(route_marked_stops)} routes serving marked stops")

    return route_marked_stops

#### traverse_route

In [107]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

def traverse_route(
    # self,
    bag_round_stop: Dict[int, Dict[int, Bag]],
    k: int,
    route_marked_stops: List[Tuple[Route, Stop]],
) -> Tuple[Dict[int, Dict[int, Bag]], List[Stop]]:
    """
    Traverse through all marked route-stops and update labels accordingly.

    :param bag_round_stop: Bag per round per stop
    :param k: current round
    :param route_marked_stops: list of marked (route, stop) for evaluation
    """

    new_marked_stops = set()

    for (marked_route, marked_stop) in route_marked_stops:
        # Traversing through route from marked stop
        route_bag = Bag()

        # Get all stops after current stop within the current route
        marked_stop_index = marked_route.stop_index(marked_stop)
        remaining_stops_in_route = marked_route.stops[marked_stop_index:]

        for stop_idx, current_stop in enumerate(remaining_stops_in_route):

            print(f"stop_idx, current_stop: {stop_idx, current_stop}")

            # Step 1: update earliest arrival times and criteria for each label L in route-bag
            update_labels = []
            for label in route_bag.labels:
                trip_stop_time = label.trip.get_stop(current_stop)

                # Take fare of previous stop in trip as fare is defined on start
                previous_stop = remaining_stops_in_route[stop_idx - 1]
                from_fare = label.trip.get_fare(previous_stop)
                ##############################################################################

                # HERE

                ##############################################################################
                # from_occuoancy = label.trip.get_stop_occupancy(previous_stop)

                label = label.update(
                    earliest_arrival_time=trip_stop_time.dts_arr,
                    fare_addition=from_fare,
                    # occupancy_addition=from_occupancy,
                )

                update_labels.append(label)
            route_bag = Bag(labels=update_labels)

            # Step 2: merge bag_route into bag_round_stop and remove dominated labels
            # The label contains the trip with which one arrives at current stop with k legs
            # and we boarded the trip at from_stop.
            bag_round_stop[k][current_stop] = bag_round_stop[k][current_stop].merge(
                route_bag
            )
            bag_update = bag_round_stop[k][current_stop].update

            # Mark stop if bag is updated
            if bag_update:
                new_marked_stops.add(current_stop)

            # Step 3: merge B_{k-1}(p) into B_r
            route_bag = route_bag.merge(bag_round_stop[k - 1][current_stop])

            # Assign trips to all newly added labels in route_bag
            # This is the trip on which we board
            update_labels = []
            for label in route_bag.labels:
                earliest_trip = marked_route.earliest_trip(
                    label.earliest_arrival_time, current_stop
                )
                if earliest_trip is not None:
                    # Update label with earliest trip in route leaving from this station
                    # If trip is different we board the trip at current_stop
                    label = label.update_trip(earliest_trip, current_stop)
                    update_labels.append(label)
            route_bag = Bag(labels=update_labels)

    logger.debug(f"{len(new_marked_stops)} reachable stops added")

    return bag_round_stop, new_marked_stops

#### get_transfer_time

In [108]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

# def get_transfer_time(self, stop_from: Stop, stop_to: Stop) -> int:
def get_transfer_time(stop_from: Stop, stop_to: Stop) -> int:
    """
    Calculate the transfer time from a stop to another stop (usually at one station)
    """
    transfers = self_timetable.transfers
    return transfers.stop_to_stop_idx[(stop_from, stop_to)].layovertime

#### add_transfer_time

In [109]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

def add_transfer_time(
    # self,
    bag_round_stop: Dict[int, Dict[Stop, Bag]],
    k: int,
    marked_stops: List[Stop],
) -> Tuple:
    """Add transfers between platforms."""

    marked_stops_transfers = set()

    # Add in transfers to other platforms
    for stop in marked_stops:
        other_station_stops = [st for st in stop.station.stops if st != stop]

        for other_stop in other_station_stops:
            # Create temp copy of B_k(p_i)
            temp_bag = Bag()
            for label in bag_round_stop[k][stop].labels:
                # Add arrival time to each label
                transfer_arrival_time = (
                    label.earliest_arrival_time
                    + get_transfer_time(stop, other_stop)
                )
                # Update label with new earliest arrival time at other_stop
                label = label.update(
                    earliest_arrival_time=transfer_arrival_time,
                    fare_addition=0,
                    from_stop=stop,
                )
                temp_bag.add(label)

            # Merg temp bag into B_k(p_j)
            bag_round_stop[k][other_stop] = bag_round_stop[k][other_stop].merge(
                temp_bag
            )
            bag_update = bag_round_stop[k][other_stop].update

            # Mark stop if bag is updated
            if bag_update:
                marked_stops_transfers.add(other_stop)

    logger.debug(f"{len(marked_stops_transfers)} transferable stops added")

    return bag_round_stop, marked_stops_transfers

#### Continue (raptor.run())

In [110]:
# Run rounds
actual_rounds = 0
for k in range(1, rounds + 1):
    logger.info(f"Analyzing possibilities round {k}")
    logger.debug(f"Stops to evaluate count: {len(marked_stops)}")

    # Copy bag from previous round
    bag_round_stop[k] = copy(bag_round_stop[k - 1])

    if len(marked_stops) > 0:
        actual_rounds = k

        # Accumulate routes serving marked stops from previous round
        route_marked_stops = accumulate_routes(marked_stops)

        # Traverse each route
        bag_round_stop, marked_stops_trips = traverse_route(
            bag_round_stop, k, route_marked_stops
        )

        # Now add footpath transfers and update
        bag_round_stop, marked_stops_transfers = add_transfer_time(
            bag_round_stop, k, marked_stops_trips
        )

        marked_stops = set(marked_stops_trips).union(marked_stops_transfers)
    else:
        break

logger.info("Finish round-based algorithm to create bag with best labels")
logger.info(f"Running time: {perf_counter() - s}")

# return bag_round_stop, actual_rounds

2022-06-10 16:39:18.121 | INFO     | __main__:<cell line: 3>:4 - Analyzing possibilities round 1
2022-06-10 16:39:18.122 | DEBUG    | __main__:<cell line: 3>:5 - Stops to evaluate count: 2
2022-06-10 16:39:18.125 | DEBUG    | __main__:accumulate_routes:19 - Found 26 routes serving marked stops
2022-06-10 16:39:18.179 | DEBUG    | __main__:traverse_route:84 - 36 reachable stops added
2022-06-10 16:39:18.184 | DEBUG    | __main__:add_transfer_time:44 - 63 transferable stops added
2022-06-10 16:39:18.185 | INFO     | __main__:<cell line: 3>:4 - Analyzing possibilities round 2
2022-06-10 16:39:18.185 | DEBUG    | __main__:<cell line: 3>:5 - Stops to evaluate count: 92
2022-06-10 16:39:18.187 | DEBUG    | __main__:accumulate_routes:19 - Found 372 routes serving marked stops


stop_idx, current_stop: (0, Stop(Arnhem Zuid-1 [2323103]))
stop_idx, current_stop: (1, Stop(Elst-1 [2323631]))
stop_idx, current_stop: (2, Stop(Nijmegen Lent-2 [2423016]))
stop_idx, current_stop: (3, Stop(Nijmegen-4b [2422126]))
stop_idx, current_stop: (4, Stop(Nijmegen Goffert-1 [2422127]))
stop_idx, current_stop: (5, Stop(Nijmegen Dukenburg-1 [2324182]))
stop_idx, current_stop: (6, Stop(Wijchen-1 [2422395]))
stop_idx, current_stop: (7, Stop(Ravenstein-1 [2324372]))
stop_idx, current_stop: (8, Stop(Oss-1 [2423375]))
stop_idx, current_stop: (9, Stop(Oss West-1 [2324247]))
stop_idx, current_stop: (10, Stop(Rosmalen-1 [2324303]))
stop_idx, current_stop: (11, Stop(Hertogenb ('s) Oost-1 [2323935]))
stop_idx, current_stop: (12, Stop(Hertogenbosch ('s)-7a [2422100]))
stop_idx, current_stop: (13, Stop(Tilburg-3 [2422152]))
stop_idx, current_stop: (14, Stop(Tilburg Universiteit-1 [2422388]))
stop_idx, current_stop: (15, Stop(Tilburg Reeshof-1 [2422386]))
stop_idx, current_stop: (16, Stop(Gilze

2022-06-10 16:39:18.522 | DEBUG    | __main__:traverse_route:84 - 173 reachable stops added
2022-06-10 16:39:18.589 | DEBUG    | __main__:add_transfer_time:44 - 258 transferable stops added
2022-06-10 16:39:18.590 | INFO     | __main__:<cell line: 3>:4 - Analyzing possibilities round 3
2022-06-10 16:39:18.591 | DEBUG    | __main__:<cell line: 3>:5 - Stops to evaluate count: 359
2022-06-10 16:39:18.601 | DEBUG    | __main__:accumulate_routes:19 - Found 834 routes serving marked stops


stop_idx, current_stop: (3, Stop(Amsterdam Amstel-1 [2323198]))
stop_idx, current_stop: (4, Stop(Amsterdam Centraal-8a [2422423]))
stop_idx, current_stop: (5, Stop(Amsterdam Sloterdijk-3 [2422032]))
stop_idx, current_stop: (6, Stop(Hoorn-1 [2323871]))
stop_idx, current_stop: (7, Stop(Hoorn Kersenboogerd-2 [2323876]))
stop_idx, current_stop: (8, Stop(Hoogkarspel-2 [2323831]))
stop_idx, current_stop: (9, Stop(Bovenkarspel-Grootebroek-2 [2423454]))
stop_idx, current_stop: (10, Stop(Bovenkarspel Flora-1 [2323337]))
stop_idx, current_stop: (11, Stop(Enkhuizen-1 [2323602]))
stop_idx, current_stop: (0, Stop(Dordrecht-1 [2323449]))
stop_idx, current_stop: (1, Stop(Rotterdam Blaak-2 [2422136]))
stop_idx, current_stop: (2, Stop(Rotterdam Centraal-9 [2427778]))
stop_idx, current_stop: (0, Stop(Dordrecht-1 [2323449]))
stop_idx, current_stop: (1, Stop(Rotterdam Blaak-2 [2422136]))
stop_idx, current_stop: (2, Stop(Rotterdam Centraal-9 [2427778]))
stop_idx, current_stop: (3, Stop(Schiedam Centrum-5 [

2022-06-10 16:39:19.237 | DEBUG    | __main__:traverse_route:84 - 340 reachable stops added
2022-06-10 16:39:19.279 | DEBUG    | __main__:add_transfer_time:44 - 298 transferable stops added
2022-06-10 16:39:19.280 | INFO     | __main__:<cell line: 3>:4 - Analyzing possibilities round 4
2022-06-10 16:39:19.280 | DEBUG    | __main__:<cell line: 3>:5 - Stops to evaluate count: 470
2022-06-10 16:39:19.288 | DEBUG    | __main__:accumulate_routes:19 - Found 795 routes serving marked stops


...

2022-06-10 16:39:19.980 | DEBUG    | __main__:traverse_route:84 - 61 reachable stops added
2022-06-10 16:39:19.986 | DEBUG    | __main__:add_transfer_time:44 - 59 transferable stops added


stop_idx, current_stop: (0, Stop(Amsterdam Centraal-14b [2323220]))
stop_idx, current_stop: (1, Stop(Amsterdam Sloterdijk-11 [2323265]))
stop_idx, current_stop: (2, Stop(Amsterdam Lelylaan-1 [2422025]))
stop_idx, current_stop: (3, Stop(Schiphol Airport-5-6 [2422149]))
stop_idx, current_stop: (4, Stop(Hoofddorp-1 [2422334]))
stop_idx, current_stop: (0, Stop(Rotterdam Centraal-3 [2324346]))
stop_idx, current_stop: (1, Stop(Delft-1 [2422319]))
stop_idx, current_stop: (2, Stop(Den Haag HS-4 [2422081]))
stop_idx, current_stop: (3, Stop(Leiden Centraal-9b [2324041]))
stop_idx, current_stop: (4, Stop(Schiphol Airport-5-6 [2422149]))
stop_idx, current_stop: (5, Stop(Amsterdam Centraal-2b [2422013]))
stop_idx, current_stop: (6, Stop(Utrecht Centraal-18 [2422163]))
stop_idx, current_stop: (0, Stop(Schiphol Airport-5-6 [2422149]))
stop_idx, current_stop: (1, Stop(Leiden Centraal-8b [2324038]))
stop_idx, current_stop: (2, Stop(Den Haag Laan v NOI-5 [2324016]))
stop_idx, current_stop: (3, Stop(Den 

2022-06-10 16:39:19.988 | INFO     | __main__:<cell line: 3>:4 - Analyzing possibilities round 5
2022-06-10 16:39:19.988 | DEBUG    | __main__:<cell line: 3>:5 - Stops to evaluate count: 100
2022-06-10 16:39:19.990 | DEBUG    | __main__:accumulate_routes:19 - Found 371 routes serving marked stops
2022-06-10 16:39:20.264 | DEBUG    | __main__:traverse_route:84 - 7 reachable stops added


stop_idx, current_stop: (0, Stop(Assen-? [2422594]))
stop_idx, current_stop: (1, Stop(Beilen-? [2422632]))
stop_idx, current_stop: (2, Stop(Hoogeveen-? [2422848]))
stop_idx, current_stop: (3, Stop(Meppel-? [2423528]))
stop_idx, current_stop: (4, Stop(Zwolle-? [2423217]))
stop_idx, current_stop: (0, Stop(Assen-? [2422594]))
stop_idx, current_stop: (1, Stop(Haren OV Transferium-? [2422877]))
stop_idx, current_stop: (2, Stop(Groningen-? [2429655]))
stop_idx, current_stop: (0, Stop(Assen-? [2422594]))
stop_idx, current_stop: (1, Stop(Haren OV Transferium-? [2422877]))
stop_idx, current_stop: (2, Stop(Groningen-? [2429655]))
stop_idx, current_stop: (0, Stop(Assen-? [2422594]))
stop_idx, current_stop: (1, Stop(Beilen-? [2422632]))
stop_idx, current_stop: (2, Stop(Hoogeveen-? [2422848]))
stop_idx, current_stop: (3, Stop(Meppel-? [2423528]))
stop_idx, current_stop: (0, Stop(Groningen-? [2429655]))
stop_idx, current_stop: (1, Stop(Haren OV Transferium-? [2422877]))
stop_idx, current_stop: (2, S

2022-06-10 16:39:20.268 | DEBUG    | __main__:add_transfer_time:44 - 5 transferable stops added
2022-06-10 16:39:20.269 | INFO     | __main__:<cell line: 30>:30 - Finish round-based algorithm to create bag with best labels
2022-06-10 16:39:20.269 | INFO     | __main__:<cell line: 31>:31 - Running time: 12.310806916997535


```python

raptor = McRaptorAlgorithm(timetable)
bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)
last_round_bag = copy(bag_round_stop[rounds])

```

Misc.

In [111]:
# Stop(Arnhem Zuid-1 [2323103])
current_stop = timetable.stations.get_stops('Arnhem Zuid')[0]
print(f"current_stop: {current_stop}")

current_label = bag_round_stop[k][current_stop].labels[0]
print(f"current_label: {current_label}")

print(f"current_label.occupancy: {current_label.occupancy}")

current_stop: Stop(Arnhem Zuid-1 [2323103])
current_label: Label(earliest_arrival_time=30600, fare=0, trip=None, from_stop=Stop(Arnhem Zuid-1 [2323103]), occupancy=0, n_trips=0, infinite=False)
current_label.occupancy: 0


In [112]:
### Continuing run_mcraptor, after raptor.run()

last_round_bag = copy(bag_round_stop[rounds])

# Calculate journets to all destinations
logger.info("Calculating journeys to all destinations")
s = perf_counter()

destination_stops = {
    st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
}
destination_stops.pop(origin_station, None)

journeys_to_destinations = dict()

2022-06-10 16:39:25.172 | INFO     | __main__:<cell line: 6>:6 - Calculating journeys to all destinations


#### best_legs_to_destination (pareto_set)

In [113]:
import numpy as np
from itertools import compress

def pareto_set(labels: List[Label], keep_equal=False):
    """
    Find the pareto-efficient points
    :param labels: list with labels
    :keep_equal return also labels with equal criteria
    :return: list with pairwise non-dominating labels
    """

    is_efficient = np.ones(len(labels), dtype=bool)
    labels_criteria = np.array([label.criteria for label in labels])
    for i, label in enumerate(labels_criteria):
        if is_efficient[i]:
            # Keep any point with a lower cost
            if keep_equal:
                # keep point with all labels equal or one lower
                is_efficient[is_efficient] = np.any(
                    labels_criteria[is_efficient] < label, axis=1
                ) + np.all(labels_criteria[is_efficient] == label, axis=1)
            else:
                is_efficient[is_efficient] = np.any(
                    labels_criteria[is_efficient] < label, axis=1
                )

            is_efficient[i] = True  # And keep self

    return list(compress(labels, is_efficient))

Misc.

In [114]:
def pareto_set2(labels: List[Label], keep_equal=False):
    """
    Find the pareto-efficient points
    :param labels: list with labels
    :keep_equal return also labels with equal criteria
    :return: list with pairwise non-dominating labels
    """

    is_efficient = np.ones(len(labels), dtype=bool)
    print(f"is_efficient: {is_efficient}")
    labels_criteria = np.array([label.criteria for label in labels])
    print(f"labels_criteria: {labels_criteria}, [earliest_arrival_time, fare, n_trips, occupancy]")
    for i, label in enumerate(labels_criteria):
        if is_efficient[i]:
            # Keep any point with a lower cost
            if keep_equal:
                # keep point with all labels equal or one lower
                is_efficient[is_efficient] = np.any(
                    labels_criteria[is_efficient] < label, axis=1
                ) + np.all(labels_criteria[is_efficient] == label, axis=1)
            else:
                is_efficient[is_efficient] = np.any(
                    labels_criteria[is_efficient] < label, axis=1
                )

            is_efficient[i] = True  # And keep self

    return list(compress(labels, is_efficient))

# Find all labels to target_stops
best_labels2 = [
    (stop, label) for stop in to_stops for label in last_round_bag[stop].labels
]

print(f"[label for (_, label) in best_labels2]: {[label for (_, label) in best_labels2]}")
print(f"pareto_set2([label for (_, label) in best_labels2]): {pareto_set2([label for (_, label) in best_labels2])}")

[label for (_, label) in best_labels2]: [Label(earliest_arrival_time=36120, fare=0, trip=Trip(hint=6931, stop_times=21), from_stop=Stop(Geldermalsen-4 [2422327]), occupancy=0, n_trips=3, infinite=False)]
is_efficient: [ True]
labels_criteria: [[36120     0     3     0]], [earliest_arrival_time, fare, n_trips, occupancy]
pareto_set2([label for (_, label) in best_labels2]): [Label(earliest_arrival_time=36120, fare=0, trip=Trip(hint=6931, stop_times=21), from_stop=Stop(Geldermalsen-4 [2422327]), occupancy=0, n_trips=3, infinite=False)]


#### continue (run_mcraptor)

In [115]:
for destination_station_name, to_stops in destination_stops.items():
    destination_legs = best_legs_to_destination_station(to_stops, last_round_bag)

    if len(destination_legs) == 0:
        logger.info("Destination unreachable with given parameters")
        continue

    journeys = reconstruct_journeys(
        from_stops, destination_legs, bag_round_stop, k=rounds
    )
    journeys_to_destinations[destination_station_name] = journeys

logger.info(f"Journey calculation time: {perf_counter() - s}")

# return journeys_to_destinations

2022-06-10 16:39:31.372 | INFO     | __main__:<cell line: 13>:13 - Journey calculation time: 6.1991955830017105


Manual implementation of `best_legs_to_destination_station()`:

In [116]:
### Create copies of variables so that we don't overwrite

destination_station_name2, to_stops2 = list(destination_stops.items())[0]

# for comparison
destination_legs2 = best_legs_to_destination_station(to_stops2, last_round_bag)

destination_station_name2, to_stops2

('Vlissingen',
 [Stop(Vlissingen-3 [2324635]),
  Stop(Vlissingen-1 [2324633]),
  Stop(Vlissingen-2 [2324634])])

Manual implementation of `best_legs_to_destination_station()`:

In [117]:
# Find all labels to target_stops
best_labels = [
    (stop, label) for stop in to_stops2 for label in last_round_bag[stop].labels
]
best_labels

[(Stop(Vlissingen-3 [2324635]),
  Label(earliest_arrival_time=42780, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Vlissingen-1 [2324633]), occupancy=0, n_trips=2, infinite=False)),
 (Stop(Vlissingen-3 [2324635]),
  Label(earliest_arrival_time=41940, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Roosendaal-4a [2324313]), occupancy=0, n_trips=3, infinite=False)),
 (Stop(Vlissingen-1 [2324633]),
  Label(earliest_arrival_time=42660, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Dordrecht-5 [2323463]), occupancy=0, n_trips=2, infinite=False)),
 (Stop(Vlissingen-1 [2324633]),
  Label(earliest_arrival_time=42060, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Vlissingen-3 [2324635]), occupancy=0, n_trips=3, infinite=False)),
 (Stop(Vlissingen-2 [2324634]),
  Label(earliest_arrival_time=42780, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Vlissingen-1 [2324633]), occupancy=0, n_trips=2, infinite=False)),
 (Stop(Vlissingen-2 

#### HERE
- We would probably want best_labels to change after first query?

Manual implementation of `best_legs_to_destination_station()`:

In [118]:
# TODO Use merge function on Bag
# Pareto optimal labels
pareto_optimal_labels = pareto_set([label for (_, label) in best_labels])
pareto_optimal_labels = [
    (stop, label) for (stop, label) in best_labels if label in pareto_optimal_labels
]
pareto_optimal_labels

[(Stop(Vlissingen-3 [2324635]),
  Label(earliest_arrival_time=41940, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Roosendaal-4a [2324313]), occupancy=0, n_trips=3, infinite=False)),
 (Stop(Vlissingen-1 [2324633]),
  Label(earliest_arrival_time=42660, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Dordrecht-5 [2323463]), occupancy=0, n_trips=2, infinite=False))]

##### IMPORTANT:
- `pareto_set` only returns labels if ***every*** criteria is better
- We will need to relax this with occupancy
    - With a cost-based method?

Manual implementation of `pareto_set()` (method called within 'best_legs_to_destination_station()`):

```python
def pareto_set(labels: List[Label], keep_equal=False):
    """
    Find the pareto-efficient points
    :param labels: list with labels
    :keep_equal return also labels with equal criteria
    :return: list with pairwise non-dominating labels
    """
```

In [119]:
######################################################################
labels = [label for (_, label) in best_labels]
keep_equal=False
######################################################################

is_efficient = np.ones(len(labels), dtype=bool)
labels_criteria = np.array([label.criteria for label in labels])
for i, label in enumerate(labels_criteria):
    if is_efficient[i]:
        # Keep any point with a lower cost
        if keep_equal:
            # keep point with all labels equal or one lower
            is_efficient[is_efficient] = np.any(
                labels_criteria[is_efficient] < label, axis=1
            ) + np.all(labels_criteria[is_efficient] == label, axis=1)
        else:
            is_efficient[is_efficient] = np.any(
                labels_criteria[is_efficient] < label, axis=1
            )

        is_efficient[i] = True  # And keep self

    # return list(compress(labels, is_efficient))

pareto_optimal_labels = list(compress(labels, is_efficient))
pareto_optimal_labels

[Label(earliest_arrival_time=41940, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Roosendaal-4a [2324313]), occupancy=0, n_trips=3, infinite=False),
 Label(earliest_arrival_time=42660, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Dordrecht-5 [2323463]), occupancy=0, n_trips=2, infinite=False)]

Manual implementation of `best_legs_to_destination_station()`:

In [120]:
pareto_optimal_labels = [
    (stop, label) for (stop, label) in best_labels if label in pareto_optimal_labels
]
pareto_optimal_labels

[(Stop(Vlissingen-3 [2324635]),
  Label(earliest_arrival_time=41940, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Roosendaal-4a [2324313]), occupancy=0, n_trips=3, infinite=False)),
 (Stop(Vlissingen-1 [2324633]),
  Label(earliest_arrival_time=42660, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Dordrecht-5 [2323463]), occupancy=0, n_trips=2, infinite=False))]

#### continue (run_mcraptor)

Manual implementation of `best_legs_to_destination_station()`:

In [121]:
"""
Continuing best_legs_to_destination_station
"""

# Label to leg, i.e. add to_stop
legs = [
    Leg(
        label.from_stop, # from_stop: Stop
        to_stop, # to_stop: Stop
        label.trip, # trip: Trip
        label.earliest_arrival_time, # earliest_arrival_time: int
        label.fare, # fare: int = 0
        # label.occupancy, # occupancy: int = 0 # 03/06/2022
        label.n_trips, # n_trips: int = 0
    )
    for (to_stop, label) in pareto_optimal_labels
]
    # return legs
destination_legs2 = legs
destination_legs2

[Leg(from_stop=Stop(Roosendaal-4a [2324313]), to_stop=Stop(Vlissingen-3 [2324635]), trip=Trip(hint=2222, stop_times=23), earliest_arrival_time=41940, fare=0, n_trips=3),
 Leg(from_stop=Stop(Dordrecht-5 [2323463]), to_stop=Stop(Vlissingen-1 [2324633]), trip=Trip(hint=2224, stop_times=17), earliest_arrival_time=42660, fare=0, n_trips=2)]

In [122]:
destination_legs2[0].occupancy, destination_legs2[0].occupancy_cost

(0.0, -0.0)

In [123]:
if len(destination_legs2) == 0:
    logger.info("Destination unreachable with given parameters")
    # continue

journeys = reconstruct_journeys(
    from_stops, destination_legs, bag_round_stop, k=rounds
)
journeys_to_destinations[destination_station_name] = journeys

logger.info(f"Journey calculation time: {perf_counter() - s}")

# return journeys_to_destinations
journeys_to_destinations

2022-06-10 16:40:38.719 | INFO     | __main__:<cell line: 10>:10 - Journey calculation time: 73.54756945802364


{'Vlissingen': [Journey(n_legs=3), Journey(n_legs=2)],
 'Dordrecht': [Journey(n_legs=1)],
 'Wijhe': [],
 'Brummen': [Journey(n_legs=1)],
 'Goes': [Journey(n_legs=2), Journey(n_legs=3)],
 'Oisterwijk': [Journey(n_legs=2)],
 'Santpoort Noord': [],
 'Amsterdam Centraal': [],
 'Vlissingen Souburg': [Journey(n_legs=2), Journey(n_legs=3)],
 'Haarlem': [Journey(n_legs=2)],
 "Harde ('t)": [],
 'Almere Oostvaarders': [],
 'Hilversum': [],
 'Amsterdam Bijlmer ArenA': [],
 'Den Haag Centraal': [Journey(n_legs=2)],
 'Maastricht': [Journey(n_legs=2)],
 'Amsterdam Sloterdijk': [],
 'Groningen Europapark': [],
 'Heiloo': [],
 'Zaltbommel': [Journey(n_legs=2)],
 'Zaandam': [],
 'Sittard': [Journey(n_legs=2)],
 'Baarn': [],
 'Vught': [Journey(n_legs=2)],
 'Abcoude': [],
 'Delft': [],
 'Rotterdam Lombardijen': [Journey(n_legs=2)],
 'Breda': [Journey(n_legs=1)],
 'Zwijndrecht': [Journey(n_legs=2)],
 'Soest Zuid': [],
 'Barendrecht': [Journey(n_legs=2)],
 'Helmond Brouwhuis': [Journey(n_legs=2), Journey(n

In [124]:
list(last_round_bag.items())[0]

(Stop(Vlissingen-3 [2324635]),
 Bag([Label(earliest_arrival_time=42780, fare=0, trip=Trip(hint=2224, stop_times=17), from_stop=Stop(Vlissingen-1 [2324633]), occupancy=0, n_trips=2, infinite=False), Label(earliest_arrival_time=41940, fare=0, trip=Trip(hint=2222, stop_times=23), from_stop=Stop(Roosendaal-4a [2324313]), occupancy=0, n_trips=3, infinite=False)], update=False))

In [128]:
# journeys_to_destinations

Can't find journey to destination station (empty) on second run

In [126]:
### Finished `run_mcprator()`

"""
# Find route between two stations
journeys_to_destinations = run_mcraptor(
    timetable,
    origin_station,
    dep_secs,
    rounds,
)

# Output journey
journeys = journeys_to_destinations[destination_station]
if len(journeys) != 0:
    for jrny in journeys:
        jrny.print(dep_secs=dep_secs)
"""

# Output journey
journeys = journeys_to_destinations[destination_station]
if len(journeys) != 0:
    for jrny in journeys:
        jrny.print(dep_secs=dep_secs)

In [129]:
# destination_station, journeys_to_destinations

We should loop through our journey and add one to occupancy

In [130]:
journeys

[]

In [131]:
journeys[0].legs

IndexError: list index out of range

In [132]:
journeys[0].occupancy() # returns the occupancy of the last leg, WRONG

IndexError: list index out of range

Add this as a method to the journey class

In [133]:
for jrny in journeys:
    for leg in jrny.legs:
        print(f"leg.trip: {leg.trip}")
        print(f"\t leg.from_stop: {leg.from_stop}")
        print(f"\t leg.occupancy: {leg.occupancy}")
        print(f"\t leg.trip.get_stop_occupancy(leg.from_stop): {leg.trip.get_stop_occupancy(leg.from_stop)}")
        print()
        leg.trip.update_occupancy(leg.from_stop)
        print(f"\t leg.occupancy: {leg.occupancy}")
        print(f"\t leg.trip.get_stop_occupancy(leg.from_stop): {leg.trip.get_stop_occupancy(leg.from_stop)}")
        print()
        print()

In [134]:
destination_legs[-1].criteria

[36120, 0, 3, 0.0]

In [100]:
assert 1==2

AssertionError: 

# .

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

---

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

<br>

# To-Do:

1. ~~We add occupancy (to instances of `TripStopTime`), do we ever remove it?~~
    - WE DON'T NEED TO REMOVE IT
        - A `TripStopTime` is a one-time instance, from one-stop to another, once throughout the day.
        - Hence it is a unique occurance.
        - We never need to remove occupancy from a 'TripStopTime'

1. ~~Ammend `occupancy()` method in `Journey` class (remane to `occupancy_cost()`)~~
    1. The `occupancy_cost()` method should yield the sum of the weighted occupancy "costs" from all of the `Legs` in the journey.
    1. Hence, we need to call the `occupancy_cost()` method in the class `Leg`. This method applies a weighted sum of the `Leg` occupancy `self.occupancy()` and the `Leg` duration (`self.dep - self.arr`).
    1.  The `occupancy_cost()` method in the `Journey` class should sum the `occupanc_cost`s for each `Leg` (`self.legs`) in the `Journey`.

1. Once we update the `occupancy`s of the `TripStopTime`s for our final journey, re-running the query doesn't work.
    - I think this is to do with the `pareto_set()` method needing ***all*** criteria to be non-dominated
    - I have manually implemented the `pareto_set()` method for a random station
    1. We should implement it for the station our journey chooses and analyse

## Questions:

1. Is a lay-over a `Trip`?
    - No, it's a `Leg` (has arrival and depature times and associated trip)

1. How do we implement the occupancy for waits? Is it handled by transfers?